In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta

In [2]:
df_data = pd.read_excel("/opt/project/src/DOE/src/function/Testcase_DOE_2-7-2024.xlsx", sheet_name="Test_Case")
df_db = pd.read_excel("/opt/project/src/DOE/src/function/Testcase_DOE_2-7-2024.xlsx", sheet_name="Prerequisite")

In [3]:
def prep_data(df):

    #Rename columns
    df.rename(columns={
                'CREATED_TIMESTAMP\n(วันที่ยื่นคำขอ)': 'CREATED_TIMESTAMP',
                'FORM_ID\n(รหัสฟอร์ม)': 'FORM_ID',
                'FORM_ID_SEQ\n(ลำดับรหัสฟอร์ม)': 'FORM_ID_SEQ',
                'ALIEN_ID\n(รหัสคนต่างด้าว)': 'ALIEN_ID',
                'ALIEN_SEQ\n(ลำดับรหัสคนต่างด้าว)': 'ALIEN_SEQ',
                'EMPLOYER_ID\n(รหัสนายจ้าง)': 'EMPLOYER_ID',
                'COMPANYNAME_TH\n(ชื่อนายจ้างไทย/สถานประกอบการ)': 'COMPANYNAME_TH',
                'COMPANYNAME_EN\n(ชื่อบริษัท (อังกฤษ))': 'COMPANYNAME_EN',
                'BUS_TYPE_ID\n(รหัสประเภทกิจการ)': 'BUS_TYPE_ID',
                'Job Description\n(ตำแหน่งงานลูกจ้าง)': 'Job_Description',
                'EMPLOYER_NO\n(เลขปชช./นิติบุคคล)': 'EMPLOYER_NO',
                'Master_Form_Type\n(FORM_TYPE_ID)': 'Master_Form_Type',
                'Master_Form_Status\n(Tracking_Status)': 'Master_Form_Status',
                   }, inplace=True)
    df.columns = [x.upper() for x in df.columns]

    # Convert columns to datetime
    df['CREATED_TIMESTAMP'] = pd.to_datetime(df['CREATED_TIMESTAMP'])
    df['CREATED_TIMESTAMP'] = df['CREATED_TIMESTAMP'].dt.date

    # Split and explode columns ALIEN_ID, ALIEN_SEQ, Job Description
    df['ALIEN_ID'] = df['ALIEN_ID'].str.split(',')
    df['ALIEN_SEQ'] = df['ALIEN_SEQ'].apply(lambda x: x.split(',') if isinstance(x, str) and ',' in x else [x])
    df['JOB_DESCRIPTION'] = df['JOB_DESCRIPTION'].str.split(',')

    # Explode the columns
    exploded_A = df.explode('ALIEN_ID').reset_index(drop=True)
    exploded_B = df.explode('ALIEN_SEQ').reset_index(drop=True)
    exploded_C = df.explode('JOB_DESCRIPTION').reset_index(drop=True)
    
    # Combine the exploded columns back into a single DataFrame
    exploded_df = exploded_A.copy()
    exploded_df['ALIEN_SEQ'] = exploded_B['ALIEN_SEQ']
    exploded_df['JOB_DESCRIPTION'] = exploded_C['JOB_DESCRIPTION']

    # Fill NaN values with previous values
    exploded_df = exploded_df.ffill()

    def remove_quote(x):
        return x.replace('"', '') if isinstance(x, str) else x

    # List of columns to process
    columns_to_process = [
        'ALIEN_ID', 'FORM_ID', 'EMPLOYER_ID', 
        'COMPANYNAME_TH', 'COMPANYNAME_EN', 
        'BUS_TYPE_ID', 'EMPLOYER_NO', 'JOB_DESCRIPTION'
    ]

    # Apply remove_quote function to each column in the list
    for column in columns_to_process:
        exploded_df[column] = exploded_df[column].apply(remove_quote)

    return exploded_df

In [4]:
data = prep_data(df_data)
db = prep_data(df_db)

In [5]:
# db = pd.read_csv('prep_prerequisite_data.csv')
# data = pd.read_csv('prep_test_case_data.csv')
selected_cols = [
    'CREATED_TIMESTAMP',
    'FORM_ID',
    'ALIEN_ID',
    'JOB_DESCRIPTION',
    'EMPLOYER_NO',
    'MASTER_FORM_TYPE',
    'MASTER_FORM_STATUS',
    'VALID_UNTIL'
    ]
db = db[selected_cols]
data = data[selected_cols]

data['CREATED_TIMESTAMP'] = pd.to_datetime(data['CREATED_TIMESTAMP'])
db['CREATED_TIMESTAMP'] = pd.to_datetime(db['CREATED_TIMESTAMP'])

In [6]:
data.head(40)

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
0,2024-04-27,00001,00001,กรรมกร,12345678,MT_59,Consider request round1,2027-04-14 11:10:59.995
1,2024-04-27,00002,00002,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
2,2024-04-27,00002,00003,งานขายของหน้าร้าน,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
3,2024-04-27,00002,00004,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
4,2024-04-27,00002,00005,งานทํามือ,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
5,2024-04-27,00002,00006,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
6,2024-04-27,00002,00007,งานขายของหน้าร้าน,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
7,2024-04-27,00002,00008,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
8,2024-04-27,00002,00009,งานทํามือ,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
9,2024-04-27,00002,00010,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000


In [7]:
db

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
0,2022-04-27,00001,00001,กรรมกร,12346803,MT_59,Consider request round1,2027-04-14 11:10:59.995
1,2024-04-15,00002,00002,กรรมกร,12346800,MT_13_EXIT,Completed,2027-04-15 11:11:00.000
2,2024-04-15,00002,00003,งานขายของหน้าร้าน,12346800,MT_13_EXIT,Completed,2027-04-15 11:11:00.000
3,2024-04-15,00002,00004,กรรมกร,12346800,MT_13_EXIT,Completed,2027-04-15 11:11:00.000
4,2024-04-15,00002,00005,งานทํามือ,12346800,MT_13_EXIT,Completed,2027-04-15 11:11:00.000
...,...,...,...,...,...,...,...,...
387,2024-05-28,00037,00385,กรรมกร,20152921,MT_13_EXIT,Completed,2027-05-02 11:10:59.995
388,2024-05-28,00037,00386,กรรมกร,20152921,MT_13_EXIT,Completed,2027-05-02 11:10:59.995
389,2024-05-28,00037,00387,กรรมกร,20152921,MT_13_EXIT,Completed,2027-05-02 11:10:59.995
390,2024-05-28,00037,00388,กรรมกร,20152921,MT_13_EXIT,Completed,2027-05-02 11:10:59.995


Flow 1

In [211]:
def check_inform_exit(data: pd.DataFrame, db: pd.DataFrame) -> tuple:
    """
    Check if a group of aliens has reported their departure from the company.

    Parameters:
    - data (pd.DataFrame): Current data.
    - db (pd.DataFrame): Historical data.

    Returns:
    - tuple: A tuple containing a string indicating overall status ('normal' or 'abnormal'),
             an integer count of abnormal entries, a list of ALIEN_IDs marked as 'normal',
             and a list of ALIEN_IDs marked as 'abnormal'.
    """
    abnormal_message =  "Aliens have not yet reported their departure from the old company."       
    abnormal_code = "TC01"

    data_filtered = data[data["MASTER_FORM_TYPE"] == "MT_59"]
    data_filtered_alien_id = data_filtered["ALIEN_ID"].to_list()
    db_filtered = db[db["ALIEN_ID"].isin(data_filtered_alien_id)]
    db_filtered['status'] = db_filtered['MASTER_FORM_TYPE'].apply(lambda x: 'normal' if 'MT_13_EXIT' in x else 'abnormal')

    data_filtered = pd.merge(data_filtered, db_filtered[['ALIEN_ID', 'status']], on='ALIEN_ID', how='left')
    data_filtered_abnormal = data_filtered[data_filtered["status"] == 'abnormal']
    data_filtered_abnormal['abnormal_desc'] = abnormal_message
    data_filtered_abnormal['abnormal_code'] = abnormal_code

    count_abnormal = data_filtered_abnormal.shape[0]
    result = 'normal' if count_abnormal < data_filtered.shape[0] else 'abnormal'

    return {'result': result,'count_abnormal': count_abnormal, 'data': data_filtered_abnormal}

#Test ID 02-05
def check_job_limits(data, config_case):
    """
    Check an employer hires more than 10 aliens for different types of work and positions.

    Parameters:
    - data (pd.DataFrame): The input data containing job information.
    - config_case (list of dict): A list of dictionaries containing job configurations with 'job' and 'number' keys.

    Returns:
    - str: 'normal' if all job counts are within limits, 'abnormal' otherwise.
    """
    
    abnormal_message =  "The employer hires more than 10 aliens for different types of work and positions."       
    abnormal_code = "TC02"
    # Filter data based on form_id and master_form_type
    data_case = data[data['MASTER_FORM_TYPE'] == 'MT_59']
    
    # Group by relevant columns and count the number of aliens per job
    data_case = data_case.groupby(['EMPLOYER_NO', 'FORM_ID', 'JOB_DESCRIPTION']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
    
    count_abnormal = 0
    job_abnormal_list = []
    # Check each job configuration
    for job_config in config_case:
        job = job_config['job']
        expected_number = job_config['number']
        
        # Calculate the total count of aliens for the specified job
        job_count = data_case[data_case['JOB_DESCRIPTION'] == job]['ALIEN_COUNT'].sum()
        # print(f'job: {job}, expected: {expected_number}, job_count: {job_count}')
        # If the job count exceeds the expected number, return 'abnormal'
        if job_count > expected_number:
            count_abnormal += 1
            job_abnormal_list.append(job)
    
    data["status"] = data["JOB_DESCRIPTION"].apply(lambda x: 'normal' if not any(item in x for item in job_abnormal_list) else 'abnormal')        
    # data["status"] = data["JOB_DESCRIPTION"].apply(lambda x: 'normal' if job_abnormal_list not in x else 'abnormal')
    data["abnormal_code"] = data['status'].apply(lambda x: 'pass' if x == 'normal' else abnormal_code)
    data["abnormal_desc"] = data['status'].apply(lambda x: 'pass' if x == 'normal' else abnormal_message)
    
    
    # If all job counts are within limits, return 'normal'
    result = 'abnormal' if "normal" not in data["status"].to_list() else 'normal'
    count_abnormal = data[data["status"] == "abnormal"].shape[0]
    return {'result': result,'count_abnormal': count_abnormal, 'job_abnormal': job_abnormal_list, 'data': data}

In [218]:
data_TC01 = data[data["FORM_ID"].isin(['00002'])]
db_TC01 = db[db["FORM_ID"].isin(['00002'])]

In [219]:
data

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
0,2024-04-27,00001,00001,กรรมกร,12345678,MT_59,Consider request round1,2027-04-14 11:10:59.995
1,2024-04-27,00002,00002,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
2,2024-04-27,00002,00003,งานขายของหน้าร้าน,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
3,2024-04-27,00002,00004,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
4,2024-04-27,00002,00005,งานทํามือ,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000
...,...,...,...,...,...,...,...,...
387,2024-07-28,00037,00385,กรรมกร,25356721,MT_59,Consider request round1,2027-05-02 11:10:59.995
388,2024-07-28,00037,00386,กรรมกร,25356721,MT_59,Consider request round1,2027-05-02 11:10:59.995
389,2024-07-28,00037,00387,กรรมกร,25356721,MT_59,Consider request round1,2027-05-02 11:10:59.995
390,2024-07-28,00037,00388,กรรมกร,25356721,MT_59,Consider request round1,2027-05-02 11:10:59.995


In [220]:
result = check_inform_exit(data_TC01, db_TC01)

In [221]:
result["data"]

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,status,abnormal_desc,abnormal_code


In [222]:
result["data"]["ALIEN_ID"].to_list()

[]

In [224]:
config_case = [
    {"job": "กรรมกร", "number": 5},
    {"job": "งานขายของหน้าร้าน", "number": 10},
    {"job": "งานทํามือ", "number": 10},
]
result = check_job_limits(filtered_data_TC01, config_case)

In [225]:
result

{'result': 'normal',
 'count_abnormal': 6,
 'job_abnormal': ['กรรมกร'],
 'data':    CREATED_TIMESTAMP FORM_ID ALIEN_ID    JOB_DESCRIPTION EMPLOYER_NO  \
 1         2024-04-27   00002    00002             กรรมกร    12345679   
 2         2024-04-27   00002    00003  งานขายของหน้าร้าน    12345679   
 3         2024-04-27   00002    00004             กรรมกร    12345679   
 4         2024-04-27   00002    00005          งานทํามือ    12345679   
 5         2024-04-27   00002    00006             กรรมกร    12345679   
 6         2024-04-27   00002    00007  งานขายของหน้าร้าน    12345679   
 7         2024-04-27   00002    00008             กรรมกร    12345679   
 8         2024-04-27   00002    00009          งานทํามือ    12345679   
 9         2024-04-27   00002    00010             กรรมกร    12345679   
 10        2024-04-27   00002    00011  งานขายของหน้าร้าน    12345679   
 11        2024-04-27   00002    00012             กรรมกร    12345679   
 
    MASTER_FORM_TYPE       MASTER_FORM_STA

In [226]:
result["data"]

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,status,abnormal_code,abnormal_desc
1,2024-04-27,00002,00002,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
2,2024-04-27,00002,00003,งานขายของหน้าร้าน,12345679,MT_59,Consider request round1,2027-04-15 11:11:00,normal,pass,pass
3,2024-04-27,00002,00004,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
4,2024-04-27,00002,00005,งานทํามือ,12345679,MT_59,Consider request round1,2027-04-15 11:11:00,normal,pass,pass
5,2024-04-27,00002,00006,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
6,2024-04-27,00002,00007,งานขายของหน้าร้าน,12345679,MT_59,Consider request round1,2027-04-15 11:11:00,normal,pass,pass
7,2024-04-27,00002,00008,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
8,2024-04-27,00002,00009,งานทํามือ,12345679,MT_59,Consider request round1,2027-04-15 11:11:00,normal,pass,pass
9,2024-04-27,00002,00010,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
10,2024-04-27,00002,00011,งานขายของหน้าร้าน,12345679,MT_59,Consider request round1,2027-04-15 11:11:00,normal,pass,pass


In [77]:
data_TC01

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
0,2024-04-27,00001,00001,กรรมกร,12345678,MT_59,Consider request round1,2027-04-14 11:10:59.995


In [76]:
db_TC01

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
0,2022-04-27,00001,00001,กรรมกร,12346803,MT_59,Consider request round1,2027-04-14 11:10:59.995


In [10]:
from flow.module import *

def flow_1(data, db, config_case):
    """
    Flow 1: Check if aliens have reported their departure and if the employer hires within the job limits.

    Parameters:
    - data (pd.DataFrame): Current data.
    - db (pd.DataFrame): Historical data.
    - config_case (list of dict): Configuration case containing job limits.

    Returns:
    - dict: A dictionary containing the status and message of the check, along with all relevant data.
    """
    # Check if aliens have reported their departure
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'data': None,
        'job_abnormal': None  # Only included if job limits check is abnormal
    }
    
    if result_check_inform_exit["result"] == "normal":
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        result_check_job_limits = check_job_limits(data[~data['ALIEN_ID'].isin(alien_ids_to_drop)], config_case)
        
        if result_check_job_limits["result"] == "normal":
            result.update({
                "status": "normal",
                "message": "Aliens have reported their departure from the old company, and the employer hires not more than 10 aliens for different types of work and positions.",
                "count_abnormal": result_check_job_limits["count_abnormal"],
                'job_abnormal': result_check_job_limits["job_abnormal"]
            })
            # Combine data from both checks
            combined_data = pd.concat([result_check_inform_exit["data"], result_check_job_limits["data"]])
            combined_data = combined_data.reset_index(drop=True)
            result['data'] = combined_data.to_dict(orient='records')
        else:
            result.update({
                "status": "abnormal",
                "message": "Aliens have reported their departure from the old company. However, the employer hires more than 10 aliens for different types of work and positions.",
                "count_abnormal": result_check_job_limits["count_abnormal"],
                'job_abnormal': result_check_job_limits["job_abnormal"]
            })
            # Correct status in data from inform exit check
            result_check_inform_exit["data"]['status'] = 'abnormal'
            result['data'] = result_check_inform_exit["data"].to_dict(orient='records')
    
    else:
        result.update({
            "status": "abnormal",
            "message": "Some aliens have not yet reported their departure from the old company.",
            "count_abnormal": result_check_inform_exit['count_abnormal']
        })
        result['data'] = result_check_inform_exit["data"].to_dict(orient='records')
    
    return result

In [26]:
def flow_1(data, db, config_case):
    """
    Flow 1: Check if aliens have reported their departure and if the employer hires within the job limits.

    Parameters:
    - data (pd.DataFrame): Current data.
    - db (pd.DataFrame): Historical data.
    - config_case (list of dict): Configuration case containing job limits.

    Returns:
    - dict: A dictionary containing the status and message of the check, along with all relevant data.
    """
    # Check if aliens have reported their departure
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'data': None,
        'job_abnormal': None  # Only included if job limits check is abnormal
    }
    
    if result_check_inform_exit["result"] == "normal":
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        result_check_job_limits = check_job_limits(data[~data['ALIEN_ID'].isin(alien_ids_to_drop)], config_case)
        
        if result_check_job_limits["result"] == "normal":
            result.update({
                "status": "normal",
                "message": "Aliens have reported their departure from the old company, and the employer hires not more than 10 aliens for different types of work and positions.",
                "count_abnormal": result_check_job_limits["count_abnormal"],
                'job_abnormal': result_check_job_limits["job_abnormal"]
            })
            # Combine data from both checks into a DataFrame
            combined_data = pd.concat([result_check_inform_exit["data"], result_check_job_limits["data"]])
            combined_data = combined_data.reset_index(drop=True)
            result['data'] = combined_data.to_dict(orient='records')  # Convert DataFrame to dict
        else:
            result.update({
                "status": "abnormal",
                "message": "Aliens have reported their departure from the old company. However, the employer hires more than 10 aliens for different types of work and positions.",
                "count_abnormal": result_check_job_limits["count_abnormal"],
                'job_abnormal': result_check_job_limits["job_abnormal"]
            })
            # Correct status in data from inform exit check
            result_check_inform_exit["data"]['status'] = 'abnormal'
            result['data'] = result_check_job_limits["data"].to_dict(orient='records')  # Convert DataFrame to dict
    
    else:
        result.update({
            "status": "abnormal",
            "message": "Some aliens have not yet reported their departure from the old company.",
            "count_abnormal": result_check_inform_exit['count_abnormal']
        })
        result['data'] = result_check_inform_exit["data"].to_dict(orient='records')  # Convert DataFrame to dict
    
    return result

In [27]:
data_TC01 = data[data["FORM_ID"].isin(['00005'])]
db_TC01 = db[db["FORM_ID"].isin(['00005'])]

config_case = [
    {"job": "กรรมกร", "number": 10},
    {"job": "งานขายของหน้าร้าน", "number": 10},
    {"job": "งานทํามือ", "number": 10},
]

In [28]:
data_TC01

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
30,2024-04-27,00005,00031,กรรมกร,12346801,MT_59,Consider request round1,2027-04-15 11:11:00
31,2024-04-27,00005,00032,กรรมกร,12346801,MT_59,Consider request round1,2027-04-15 11:11:00
32,2024-04-27,00005,00033,กรรมกร,12346801,MT_59,Consider request round1,2027-04-15 11:11:00
33,2024-04-27,00005,00034,กรรมกร,12346801,MT_59,Consider request round1,2027-04-15 11:11:00
34,2024-04-27,00005,00035,กรรมกร,12346801,MT_59,Consider request round1,2027-04-15 11:11:00
35,2024-04-27,00005,00036,กรรมกร,12346801,MT_59,Consider request round1,2027-04-15 11:11:00
36,2024-04-27,00005,00037,กรรมกร,12346801,MT_59,Consider request round1,2027-04-15 11:11:00
37,2024-04-27,00005,00038,กรรมกร,12346801,MT_59,Consider request round1,2027-04-15 11:11:00
38,2024-04-27,00005,00039,กรรมกร,12346801,MT_59,Consider request round1,2027-04-15 11:11:00
39,2024-04-27,00005,00040,กรรมกร,12346801,MT_59,Consider request round1,2027-04-15 11:11:00


In [29]:
db_TC01

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
30,2024-04-15,00005,00031,กรรมกร,18152918,MT_13_EXIT,Completed,2027-04-15 11:11:00
31,2024-04-15,00005,00032,กรรมกร,18152918,MT_13_EXIT,Completed,2027-04-15 11:11:00
32,2024-04-15,00005,00033,กรรมกร,18152918,MT_13_EXIT,Completed,2027-04-15 11:11:00
33,2024-04-15,00005,00034,กรรมกร,18152918,MT_13_EXIT,Completed,2027-04-15 11:11:00
34,2024-04-15,00005,00035,กรรมกร,18152918,MT_13_EXIT,Completed,2027-04-15 11:11:00
35,2024-04-15,00005,00036,กรรมกร,18152918,MT_13_EXIT,Completed,2027-04-15 11:11:00
36,2024-04-15,00005,00037,กรรมกร,18152918,MT_13_EXIT,Completed,2027-04-15 11:11:00
37,2024-04-15,00005,00038,กรรมกร,18152918,MT_13_EXIT,Completed,2027-04-15 11:11:00
38,2024-04-15,00005,00039,กรรมกร,18152918,MT_13_EXIT,Completed,2027-04-15 11:11:00
39,2024-04-15,00005,00040,กรรมกร,18152918,MT_13_EXIT,Completed,2027-04-15 11:11:00


In [30]:
result = flow_1(data_TC01, db_TC01, config_case)

   CREATED_TIMESTAMP FORM_ID ALIEN_ID JOB_DESCRIPTION EMPLOYER_NO  \
30        2024-04-27   00005    00031          กรรมกร    12346801   
31        2024-04-27   00005    00032          กรรมกร    12346801   
32        2024-04-27   00005    00033          กรรมกร    12346801   
33        2024-04-27   00005    00034          กรรมกร    12346801   
34        2024-04-27   00005    00035          กรรมกร    12346801   
35        2024-04-27   00005    00036          กรรมกร    12346801   
36        2024-04-27   00005    00037          กรรมกร    12346801   
37        2024-04-27   00005    00038          กรรมกร    12346801   
38        2024-04-27   00005    00039          กรรมกร    12346801   
39        2024-04-27   00005    00040          กรรมกร    12346801   
40        2024-04-27   00005    00041          กรรมกร    12346801   

   MASTER_FORM_TYPE       MASTER_FORM_STATUS         VALID_UNTIL    status  \
30            MT_59  Consider request round1 2027-04-15 11:11:00  abnormal   
31            M

In [31]:
result

{'status': 'abnormal',
 'message': 'Aliens have reported their departure from the old company. However, the employer hires more than 10 aliens for different types of work and positions.',
 'count_abnormal': 11,
 'data': [{'CREATED_TIMESTAMP': Timestamp('2024-04-27 00:00:00'),
   'FORM_ID': '00005',
   'ALIEN_ID': '00031',
   'JOB_DESCRIPTION': 'กรรมกร',
   'EMPLOYER_NO': '12346801',
   'MASTER_FORM_TYPE': 'MT_59',
   'MASTER_FORM_STATUS': 'Consider request round1',
   'VALID_UNTIL': Timestamp('2027-04-15 11:11:00'),
   'status': 'abnormal',
   'abnormal_code': 'TC02',
   'abnormal_desc': 'The employer hires more than 10 aliens for different types of work and positions.'},
  {'CREATED_TIMESTAMP': Timestamp('2024-04-27 00:00:00'),
   'FORM_ID': '00005',
   'ALIEN_ID': '00032',
   'JOB_DESCRIPTION': 'กรรมกร',
   'EMPLOYER_NO': '12346801',
   'MASTER_FORM_TYPE': 'MT_59',
   'MASTER_FORM_STATUS': 'Consider request round1',
   'VALID_UNTIL': Timestamp('2027-04-15 11:11:00'),
   'status': 'ab

In [33]:
result["data"]

[{'CREATED_TIMESTAMP': Timestamp('2024-04-27 00:00:00'),
  'FORM_ID': '00005',
  'ALIEN_ID': '00031',
  'JOB_DESCRIPTION': 'กรรมกร',
  'EMPLOYER_NO': '12346801',
  'MASTER_FORM_TYPE': 'MT_59',
  'MASTER_FORM_STATUS': 'Consider request round1',
  'VALID_UNTIL': Timestamp('2027-04-15 11:11:00'),
  'status': 'abnormal',
  'abnormal_code': 'TC02',
  'abnormal_desc': 'The employer hires more than 10 aliens for different types of work and positions.'},
 {'CREATED_TIMESTAMP': Timestamp('2024-04-27 00:00:00'),
  'FORM_ID': '00005',
  'ALIEN_ID': '00032',
  'JOB_DESCRIPTION': 'กรรมกร',
  'EMPLOYER_NO': '12346801',
  'MASTER_FORM_TYPE': 'MT_59',
  'MASTER_FORM_STATUS': 'Consider request round1',
  'VALID_UNTIL': Timestamp('2027-04-15 11:11:00'),
  'status': 'abnormal',
  'abnormal_code': 'TC02',
  'abnormal_desc': 'The employer hires more than 10 aliens for different types of work and positions.'},
 {'CREATED_TIMESTAMP': Timestamp('2024-04-27 00:00:00'),
  'FORM_ID': '00005',
  'ALIEN_ID': '0003

In [32]:
type(result["data"])

list

In [25]:
type(result["data"].to_dict())

dict

In [294]:
df_test = pd.DataFrame(result['data'])

In [295]:
df_test

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,status,abnormal_desc,abnormal_code
0,2024-04-27,00001,00001,กรรมกร,12345678,MT_59,Consider request round1,2027-04-14 11:10:59.995,abnormal,Aliens have not yet reported their departure f...,TC01
1,2024-04-27,00002,00002,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,abnormal,The employer hires more than 10 aliens for dif...,TC02
2,2024-04-27,00002,00003,งานขายของหน้าร้าน,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,normal,pass,pass
3,2024-04-27,00002,00004,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,abnormal,The employer hires more than 10 aliens for dif...,TC02
4,2024-04-27,00002,00005,งานทํามือ,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,normal,pass,pass
5,2024-04-27,00002,00006,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,abnormal,The employer hires more than 10 aliens for dif...,TC02
6,2024-04-27,00002,00007,งานขายของหน้าร้าน,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,normal,pass,pass
7,2024-04-27,00002,00008,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,abnormal,The employer hires more than 10 aliens for dif...,TC02
8,2024-04-27,00002,00009,งานทํามือ,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,normal,pass,pass
9,2024-04-27,00002,00010,กรรมกร,12345679,MT_59,Consider request round1,2027-04-15 11:11:00.000,abnormal,The employer hires more than 10 aliens for dif...,TC02


In [274]:
result['data']

[{'CREATED_TIMESTAMP': Timestamp('2024-04-27 00:00:00'),
  'FORM_ID': '00001',
  'ALIEN_ID': '00001',
  'JOB_DESCRIPTION': 'กรรมกร',
  'EMPLOYER_NO': '12345678',
  'MASTER_FORM_TYPE': 'MT_59',
  'MASTER_FORM_STATUS': 'Consider request round1',
  'VALID_UNTIL': Timestamp('2027-04-14 11:10:59.995000'),
  'status': 'abnormal',
  'abnormal_desc': 'Aliens have not yet reported their departure from the old company.',
  'abnormal_code': 'TC01'}]

# Flow 4

In [320]:
def check_relocate_condition_from_B(data, df, limit_count, limit_days):
    """
    Check if a group of aliens moved to location B within the limit of people and have been relocated for more than a specified number of days. 

    Parameters:
    - df (pd.DataFrame): DataFrame containing relocation data with 'CREATED_TIMESTAMP' and 'ALIEN_COUNT' columns.
    - end_date (str or pd.Timestamp): The end date for the check in 'YYYY-MM-DD' format or as a Timestamp.
    - limit_count (int): The limit of people allowed to relocate.
    - limit_days (int): The number of days to check for relocation.

    Returns:
    - str: A message indicating whether the condition is normal or abnormal.
    """
    count_abnormal = 0
    df.sort_values(by = "CREATED_TIMESTAMP", ascending= False, inplace= True)
    end_date = df.iloc[0]['CREATED_TIMESTAMP']
    # Ensure CREATED_TIMESTAMP is in datetime format
    df['CREATED_TIMESTAMP'] = pd.to_datetime(df['CREATED_TIMESTAMP'])

    # If end_date is a Timestamp, convert it to datetime; otherwise, parse it
    if isinstance(end_date, pd.Timestamp) or isinstance(end_date, pd.DatetimeIndex):
        end_date = end_date.to_pydatetime()
    else:
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    # Calculate the start date for the two-month timeframe
    start_date_time_frame = end_date - timedelta(days=90)
    # print(start_date_time_frame)
    # Filter the DataFrame for the two-month timeframe
    df_time_frame = df[(df['CREATED_TIMESTAMP'] >= start_date_time_frame) & (df['CREATED_TIMESTAMP'] <= end_date)]
    # print(df_time_frame)
    # Calculate the total count within the two-month timeframe
    total_count_time_frame = df_time_frame['ALIEN_COUNT'].sum()
    # print(total_count_time_frame)

    # Check if the total count within the two-month timeframe exceeds the limit_count
    if total_count_time_frame > limit_count:
        # Calculate the start date by subtracting limit_days from end date
        start_date = end_date - timedelta(days=limit_days - 1)
        
        # Filter the DataFrame between the calculated start date and the end date
        filtered_df = df[(df['CREATED_TIMESTAMP'] >= start_date) & (df['CREATED_TIMESTAMP'] <= end_date)]
        
        # Calculate total count and number of days within the limit_days timeframe
        total_count = filtered_df['ALIEN_COUNT'].sum()
        total_days = (filtered_df['CREATED_TIMESTAMP'].max() - filtered_df['CREATED_TIMESTAMP'].min()).days + 1
        # print(filtered_df)
        # print(total_count)
        # Check scenarios
        if total_count < limit_count:
            data["status"] = "normal"
            data["abnormal_code"] = "pass"
            data["abnormal_desc"] = "pass"
        else:
            if total_days <= limit_days:
                data["status"] = "abnormal"
                data["abnormal_code"] = "TC02"
                data["abnormal_desc"] = "Aliens moved to B exceeding the limit of people and have been relocated for more than a specified number of days."
                count_abnormal = total_count
            else:
                data["status"] = "normal"
                data["abnormal_code"] = "pass"
                data["abnormal_desc"] = "pass"
    else:
        data["status"] = "normal"
        data["abnormal_code"] = "pass"
        data["abnormal_desc"] = "pass"
        
    result = 'normal' if count_abnormal < data.shape[0] else 'abnormal'    
    total_relocate_day = "pass" if result == "normal" else total_days
    return {'result': result,'count_abnormal': count_abnormal, 'total_relocate_day': total_relocate_day, 'data': data}

In [321]:
config_case = {
    "number": 50,
    "day": 20
    }

data_TC20 = data[data["FORM_ID"].isin(['00032', '00033', '00034'])]
db_TC20 = db[db["FORM_ID"].isin(['00032', '00033', '00034'])]
data_case = data_TC20.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
db_case = db_TC20.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
concat_df = pd.concat([db_case, data_case], axis=0).reset_index(drop=True)

In [322]:
result = check_relocate_condition_from_B(data_TC20, concat_df, config_case["number"], config_case["day"])

/tmp/ipykernel_47964/3100570731.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["status"] = "abnormal"
/tmp/ipykernel_47964/3100570731.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["abnormal_code"] = "TC02"
/tmp/ipykernel_47964/3100570731.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [324]:
result

{'result': 'abnormal',
 'count_abnormal': 51,
 'total_relocate_day': 20,
 'data':     CREATED_TIMESTAMP FORM_ID ALIEN_ID   JOB_DESCRIPTION EMPLOYER_NO  \
 290        2024-07-01   00032    00288            กรรมกร    33152924   
 291        2024-07-01   00032    00289            กรรมกร    33152924   
 292        2024-07-01   00032    00290            กรรมกร    33152924   
 293        2024-07-01   00032    00291            กรรมกร    33152924   
 294        2024-07-01   00032    00292            กรรมกร    33152924   
 295        2024-07-01   00032    00293            กรรมกร    33152924   
 296        2024-07-01   00032    00294            กรรมกร    33152924   
 297        2024-07-01   00032    00295            กรรมกร    33152924   
 298        2024-07-01   00032    00296            กรรมกร    33152924   
 299        2024-07-01   00032    00297            กรรมกร    33152924   
 300        2024-07-01   00032    00298            กรรมกร    33152924   
 301        2024-07-01   00032    00299    

In [334]:
def flow_4(data, db, config_case):
    """
    Flow 4: Check relocation conditions and departure reporting status of aliens.

    Parameters:
    - data (pd.DataFrame): Current data.
    - db (pd.DataFrame): Historical data.
    - config_case (dict): Configuration case containing number and day limits.

    Returns:
    - dict: A dictionary containing the status, message, count of abnormalities, total relocate days, and all relevant data.
    """
    messages = {
        "normal": "Aliens moved to B not exceeding the limit of people and have been relocated for less than a specified number of days.",
        "abnormal_relocate_B": "Aliens moved to B exceeding the limit of people and have been relocated for more than a specified number of days.",
        "check_inform_exit": "Aliens have not yet reported their departure from the old company but have already applied for the new one."
    }
    
    # Check if aliens have reported their departure
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'total_relocate_day': 0,
        'data': None
    }
    
    if result_check_inform_exit["result"] == "normal":
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        data_case = data[~data['ALIEN_ID'].isin(alien_ids_to_drop)]
        db_case = db[~db['ALIEN_ID'].isin(alien_ids_to_drop)]
        
        # Create concat_df
        data_case_agg = data_case.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
        db_case_agg = db_case.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
        concat_df = pd.concat([db_case_agg, data_case_agg], axis=0).reset_index(drop=True)
        
        result_check_relocate_condition_from_B = check_relocate_condition_from_B(data_case, concat_df, config_case["number"], config_case["day"])
        
        if result_check_relocate_condition_from_B["result"] == "normal":
            result.update({
                "status": "normal", 
                "message": messages["normal"], 
                "count_abnormal": result_check_relocate_condition_from_B['count_abnormal'],
                "total_relocate_day": result_check_relocate_condition_from_B['total_relocate_day'],
                "data": pd.concat([result_check_inform_exit['data'], result_check_relocate_condition_from_B["data"]], ignore_index=True)
            })
        else:
            result.update({
                "status": "abnormal", 
                "message": messages["abnormal_relocate_B"],
                "count_abnormal": result_check_relocate_condition_from_B['count_abnormal'],
                "total_relocate_day": result_check_relocate_condition_from_B['total_relocate_day'],
                "data": pd.concat([result_check_inform_exit['data'], result_check_relocate_condition_from_B["data"]], ignore_index=True)
            })
    
    else:
        result.update({
            "status": "abnormal", 
            "message": messages["check_inform_exit"],
            "count_abnormal": result_check_inform_exit['count_abnormal'],
            "data": result_check_inform_exit['data']
        })
    
    return result

In [339]:
config_case = {
    "number": 50,
    "day": 20
    }

data_TC20 = data[data["FORM_ID"].isin(['00035', '00036', '00037'])]
db_TC20 = db[db["FORM_ID"].isin(['00035', '00036', '00037'])]

In [340]:
result = flow_4(data_TC20, db_TC20, config_case)

In [341]:
result

{'status': 'normal',
 'message': 'Aliens moved to B not exceeding the limit of people and have been relocated for less than a specified number of days.',
 'count_abnormal': 0,
 'total_relocate_day': 'pass',
 'data':    CREATED_TIMESTAMP FORM_ID ALIEN_ID JOB_DESCRIPTION EMPLOYER_NO  \
 0         2024-06-27   00035    00339          กรรมกร    25356721   
 1         2024-06-27   00035    00340          กรรมกร    25356721   
 2         2024-06-27   00035    00341          กรรมกร    25356721   
 3         2024-06-27   00035    00342          กรรมกร    25356721   
 4         2024-06-27   00035    00343          กรรมกร    25356721   
 5         2024-06-27   00035    00344          กรรมกร    25356721   
 6         2024-06-27   00035    00345          กรรมกร    25356721   
 7         2024-06-27   00035    00346          กรรมกร    25356721   
 8         2024-06-27   00035    00347          กรรมกร    25356721   
 9         2024-06-27   00035    00348          กรรมกร    25356721   
 10        2024

In [342]:
result['data']

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,status,abnormal_desc,abnormal_code
0,2024-06-27,00035,00339,กรรมกร,25356721,MT_59,Consider request round1,2027-04-30 11:10:59.995,normal,pass,pass
1,2024-06-27,00035,00340,กรรมกร,25356721,MT_59,Consider request round1,2027-04-30 11:10:59.995,normal,pass,pass
2,2024-06-27,00035,00341,กรรมกร,25356721,MT_59,Consider request round1,2027-04-30 11:10:59.995,normal,pass,pass
3,2024-06-27,00035,00342,กรรมกร,25356721,MT_59,Consider request round1,2027-04-30 11:10:59.995,normal,pass,pass
4,2024-06-27,00035,00343,กรรมกร,25356721,MT_59,Consider request round1,2027-04-30 11:10:59.995,normal,pass,pass
5,2024-06-27,00035,00344,กรรมกร,25356721,MT_59,Consider request round1,2027-04-30 11:10:59.995,normal,pass,pass
6,2024-06-27,00035,00345,กรรมกร,25356721,MT_59,Consider request round1,2027-04-30 11:10:59.995,normal,pass,pass
7,2024-06-27,00035,00346,กรรมกร,25356721,MT_59,Consider request round1,2027-04-30 11:10:59.995,normal,pass,pass
8,2024-06-27,00035,00347,กรรมกร,25356721,MT_59,Consider request round1,2027-04-30 11:10:59.995,normal,pass,pass
9,2024-06-27,00035,00348,กรรมกร,25356721,MT_59,Consider request round1,2027-04-30 11:10:59.995,normal,pass,pass


TC20

In [202]:
config_case = {
    "number": 50,
    "day": 20
    }

In [203]:
data_TC20 = data[data["FORM_ID"].isin(['00032', '00033', '00034'])]
db_TC20 = db[db["FORM_ID"].isin(['00032', '00033', '00034'])]

In [204]:
data_TC20.shape

(51, 8)

In [205]:
data_case = data_TC20.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
db_case = db_TC20.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()

In [206]:
data_case

,CREATED_TIMESTAMP,EMPLOYER_NO,FORM_ID,ALIEN_COUNT
0,2024-06-30,33152924,00032,15
1,2024-07-09,33152924,00033,16
2,2024-07-20,33152924,00034,20


In [207]:
db_case

,CREATED_TIMESTAMP,EMPLOYER_NO,FORM_ID,ALIEN_COUNT
0,2024-05-09,12345700,00033,16
1,2024-05-20,23095701,00034,20
2,2024-05-30,12346993,00032,15


In [208]:
concat_df = pd.concat([db_case, data_case], axis=0).reset_index(drop=True)

In [209]:
concat_df

,CREATED_TIMESTAMP,EMPLOYER_NO,FORM_ID,ALIEN_COUNT
0,2024-05-09,12345700,00033,16
1,2024-05-20,23095701,00034,20
2,2024-05-30,12346993,00032,15
3,2024-06-30,33152924,00032,15
4,2024-07-09,33152924,00033,16
5,2024-07-20,33152924,00034,20


In [210]:
concat_df.sort_values(by = "CREATED_TIMESTAMP", ascending= False, inplace= True)

In [211]:
concat_df

,CREATED_TIMESTAMP,EMPLOYER_NO,FORM_ID,ALIEN_COUNT
5,2024-07-20,33152924,00034,20
4,2024-07-09,33152924,00033,16
3,2024-06-30,33152924,00032,15
2,2024-05-30,12346993,00032,15
1,2024-05-20,23095701,00034,20
0,2024-05-09,12345700,00033,16


In [13]:
# Define the check_scenario function
def check_scenario(df, limit_count, limit_days):
    
    df.sort_values(by = "CREATED_TIMESTAMP", ascending= False, inplace= True)
    end_date = df.iloc[0]['CREATED_TIMESTAMP']
    # Ensure CREATED_TIMESTAMP is in datetime format
    df['CREATED_TIMESTAMP'] = pd.to_datetime(df['CREATED_TIMESTAMP'])

    # If end_date is a Timestamp, convert it to datetime; otherwise, parse it
    if isinstance(end_date, pd.Timestamp) or isinstance(end_date, pd.DatetimeIndex):
        end_date = end_date.to_pydatetime()
    else:
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    # Calculate the start date for the two-month timeframe
    start_date_time_frame = end_date - timedelta(days=90)
    print(start_date_time_frame)
    # Filter the DataFrame for the two-month timeframe
    df_time_frame = df[(df['CREATED_TIMESTAMP'] >= start_date_time_frame) & (df['CREATED_TIMESTAMP'] <= end_date)]
    print(df_time_frame)
    # Calculate the total count within the two-month timeframe
    total_count_time_frame = df_time_frame['ALIEN_COUNT'].sum()
    print(total_count_time_frame)

    # Check if the total count within the two-month timeframe exceeds the limit_count
    if total_count_time_frame > limit_count:
        # Calculate the start date by subtracting limit_days from end date
        start_date = end_date - timedelta(days=limit_days - 1)
        
        # Filter the DataFrame between the calculated start date and the end date
        filtered_df = df[(df['CREATED_TIMESTAMP'] >= start_date) & (df['CREATED_TIMESTAMP'] <= end_date)]
        
        # Calculate total count and number of days within the limit_days timeframe
        total_count = filtered_df['ALIEN_COUNT'].sum()
        total_days = (filtered_df['CREATED_TIMESTAMP'].max() - filtered_df['CREATED_TIMESTAMP'].min()).days + 1
        print(filtered_df)
        print(total_count)
        # Check scenarios
        if total_count < limit_count:
            return "ปกติ: Count exceeds limit but days are within limit"
        else:
            if total_days <= limit_days:
                return "ผิดปกติ: Count exceeds limit but days are within limit"
            else:
                return "ปกติ: Count exceeds limit and days exceed limit"
    else:
        return "ปกติ: Total count within two months is within limit"

In [213]:
anomaly = check_scenario(concat_df, config_case['number'], config_case['day'])
anomaly

2024-04-21 00:00:00
  CREATED_TIMESTAMP EMPLOYER_NO FORM_ID  ALIEN_COUNT
5        2024-07-20    33152924   00034           20
4        2024-07-09    33152924   00033           16
3        2024-06-30    33152924   00032           15
2        2024-05-30    12346993   00032           15
1        2024-05-20    23095701   00034           20
0        2024-05-09    12345700   00033           16
102
  CREATED_TIMESTAMP EMPLOYER_NO FORM_ID  ALIEN_COUNT
5        2024-07-20    33152924   00034           20
4        2024-07-09    33152924   00033           16
36


'ปกติ: Count exceeds limit but days are within limit'

# flow 2 

In [19]:
data_TC7 = data[data["FORM_ID"].isin(["00006"])]
db_TC7 = db[db["FORM_ID"].isin(["00006"])]

In [20]:
data_TC7

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
41,2024-06-27,00006,00501,กรรมกร,12346801,MT_59,Consider request round1,2024-07-15 11:11:00


In [21]:
db_TC7

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
41,2024-06-27,00006,00501,กรรมกร,177346803,MT_13_EXIT,Completed,2024-07-15 11:11:00


In [61]:
'normal' if "MT_13_EXIT" not in db_TC7['MASTER_FORM_TYPE'].values else 'abnormal'

'abnormal'

In [57]:
db_TC6["EMPLOYER_NO"].to_list()[0]

'177346803'

In [62]:
from module import *
def flow_2(data, db, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B):
    """
    Flow 2: Check various conditions for aliens moving from employer A to employer B.

    Parameters:
    - data (pd.DataFrame): Current data.
    - db (pd.DataFrame): Historical data.
    - config_case (dict): Configuration case containing 'number' and 'day' keys.
    - EMPLOYER_NO_A (str): Employer number for location A.
    - EMPLOYER_NO_B (str): Employer number for location B.

    Returns:
    - dict: A dictionary containing the status and message of the check.
    """
    # Define messages
    messages = {
        "normal": "Aliens moved from A to B not exceeding the limit of people and have been relocated for less than a specified number of days.",
        "abnormal_relocate": "Aliens moved from A to B exceeding the limit of people and have been relocated for more than a specified number of days.",
        "abnormal_resign_B": "Aliens has moved out of the company but has not yet reported.",
        "abnormal_resign_A": "Aliens has not yet reported their departure from the old company but has already applied for the new one.",
        "abnormal_expire": "The application submission date and the expiration date of the work permit is less than or equal to 30 days."
    }

    # Check conditions in sequence
    if check_status_resign_b(db) != "normal":
        return {"status": "abnormal", "message": messages["abnormal_resign_B"]}
    
    if check_status_resign_a(data, db) != "normal":
        return {"status": "abnormal", "message": messages["abnormal_resign_A"]}
    
    if check_relocate_condition_from_A_to_B(data, db, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B) != "normal":
        return {"status": "abnormal", "message": messages["abnormal_relocate"]}
    
    if check_expire_condition(data, db) == "normal":
        return {"status": "normal", "message": messages["normal"]}
    
    return {"status": "abnormal", "message": messages["abnormal_expire"]}

In [63]:
config_case = {
    "number": 20,
    "day": 14
    }
flow_2(data_TC7, db_TC7, config_case, db_TC7["EMPLOYER_NO"].to_list()[0], data_TC7["EMPLOYER_NO"].to_list()[0])

1


{'status': 'abnormal',
 'message': 'The application submission date and the expiration date of the work permit is less than or equal to 30 days.'}

In [137]:
def check_expire_condition(data, db):
    """
    remaining period between the application submission date and the expiration date of the work permit is less than or equal to 30 days.

    Parameters:
    - data (pd.DataFrame): The input data containing job information.
    - db (pd.DataFrame): The database containing exit information.

    Returns:
    - str: 'normal' if the timestamp condition is met, 'abnormal' otherwise.
    """
    # Filter data based on master_form_type
    data_case = data[data['MASTER_FORM_TYPE'] == 'MT_59']
    db_case = db[db['MASTER_FORM_TYPE'] == 'MT_13_EXIT']
    
    # Merge data and db on 'ALIEN_ID'
    merged_id = pd.merge(data_case, db_case, on='ALIEN_ID', how='inner')
    
    # Convert timestamps to datetime
    merged_id['CREATED_TIMESTAMP_x'] = pd.to_datetime(merged_id['CREATED_TIMESTAMP_x'])
    merged_id['VALID_UNTIL_y'] = pd.to_datetime(merged_id['VALID_UNTIL_y'])
    
    # Check timestamp condition
    check_anomaly = (merged_id['CREATED_TIMESTAMP_x'] + pd.Timedelta(days=30)) < merged_id['VALID_UNTIL_y']
    print(check_anomaly)
    
    # Determine anomaly status
    return 'abnormal' if not check_anomaly.iloc[0] else 'normal'

def check_status_resign_a(data, db):
    """
    Check aliens has not yet reported their departure from the old company but has already applied for the new one.

    Parameters:
    - data (pd.DataFrame): The input data containing job information.
    - db (pd.DataFrame): The database containing status information.

    Returns:
    - str: 'normal' if the required status is present, 'abnormal' otherwise.
    """
    # Filter data based on master_form_type
    data_case = data[data['MASTER_FORM_TYPE'] == 'MT_59']
    
    # Merge data and db on 'ALIEN_ID'
    merged_id = pd.merge(data_case, db, on='ALIEN_ID', how='inner',suffixes=('_A', '_B'))
    
    # Check if the required status is present in the merged data
    return 'abnormal' if 'MT_13_EXIT' not in merged_id['MASTER_FORM_STATUS_B'].values else 'normal'


#Test ID 8 ***
def check_status_resign_b(db):
    """
    Check aliens has moved out of the company but has not yet reported their arrival at the new place.

    Parameters:
    - db (pd.DataFrame): The database containing status information.

    Returns:
    - str: 'normal' if the required status is not present, 'abnormal' otherwise.
    """
    # Check if the required status is present in the db
    return 'normal' if "MT_13_EXIT" not in db['MASTER_FORM_STATUS'].values else 'abnormal'


def check_relocate_condition_from_A_to_B(data, db, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B):
    """
    Check if a group of aliens moved from A to B exceeding the limit of people and have been relocated for more than a specified number of days.

    Parameters:
    - data (pd.DataFrame): DataFrame containing access preparation data.
    - db (pd.DataFrame): DataFrame containing exit preparation data.
    - config_case (dict): Configuration case containing 'number' and 'day' keys.
    - EMPLOYER_NO_A (str): Employer number for location A.
    - EMPLOYER_NO_B (str): Employer number for location B.

    Returns:
    - tuple: A tuple containing the anomaly status and the merged DataFrame.
    """
    # Filter for MT_13_EXIT at location A
    db_exit_filter = db[(db["EMPLOYER_NO"] == EMPLOYER_NO_A) & (db["MASTER_FORM_TYPE"] == "MT_13_EXIT")]

    # Filter for MT_59 at location B
    db_access_filter = data[(data["EMPLOYER_NO"] == EMPLOYER_NO_B) & (data["MASTER_FORM_TYPE"] == "MT_59")]

    # Merge on ALIEN_ID
    merged_alien_id = pd.merge(db_exit_filter, db_access_filter, on='ALIEN_ID', how='inner', suffixes=('_A', '_B'))
    print(merged_alien_id)
    # Group by date and employer numbers
    count_alien_date = merged_alien_id.groupby(['CREATED_TIMESTAMP_A', "EMPLOYER_NO_A", "EMPLOYER_NO_B"]).agg(
        ALIEN_COUNT=('ALIEN_ID', 'count')
    ).reset_index().rename(columns={'CREATED_TIMESTAMP_A': 'CREATED_TIMESTAMP'})
    
    print(count_alien_date)
        
    # Check relocation condition
    anomaly = check_scenario(
        count_alien_date, 
        count_alien_date.iloc[-1]['CREATED_TIMESTAMP'], 
        config_case['number'], 
        config_case['day']
    )
    
    return anomaly

In [11]:
check_relocate_condition_from_A_to_B(data_TC09, db_TC09, config_case, "13155987", "12346803")

10


'normal'

TC08

In [69]:
data_TC8 = data[data["FORM_ID"].isin(["00008"])]
db_TC8 = db[db["FORM_ID"].isin(["00008"])]

columns_to_nan = ['ALIEN_ID', 'JOB_DESCRIPTION', 'EMPLOYER_NO']
data_TC8[columns_to_nan] = np.nan


/tmp/ipykernel_47964/2094565483.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_TC8[columns_to_nan] = np.nan


In [70]:
data_TC8

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
45,2024-06-27,00008,NaN,NaN,NaN,MT_59,Consider request round1,2027-04-15 11:11:00


In [71]:
db_TC8

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
45,2024-02-27,00008,00505,งานทํารองเท้า,12346803,MT_13_EXIT,Completed,2027-04-15 11:11:00


In [72]:
def check_status_resign_a(data, db):
    """
    Check aliens has not yet reported their departure from the old company but has already applied for the new one.

    Parameters:
    - data (pd.DataFrame): The input data containing job information.
    - db (pd.DataFrame): The database containing status information.

    Returns:
    - str: 'normal' if the required status is present, 'abnormal' otherwise.
    """
    # Filter data based on master_form_type
    data_case = data[data['MASTER_FORM_TYPE'] == 'MT_59']
    
    # Merge data and db on 'ALIEN_ID'
    merged_id = pd.merge(data_case, db, on='ALIEN_ID', how='inner',suffixes=('_A', '_B'))
    
    # Check if the required status is present in the merged data
    return 'abnormal' if 'MT_13_EXIT' not in merged_id['MASTER_FORM_TYPE_B'].values else 'normal'

In [73]:
check_status_resign_a(data_TC8, db_TC8)

ValueError: You are trying to merge on float64 and object columns for key 'ALIEN_ID'. If you wish to proceed you should use pd.concat

In [58]:
#  group by alein id in db
#  select col create timestemp, master form type
#  sort by create timestemp 
# select latest timestemp and check mt59 or mt-exit


# Ensure CREATED_TIMESTAMP is in datetime format
db_TC8['CREATED_TIMESTAMP'] = pd.to_datetime(db_TC8['CREATED_TIMESTAMP'])

# Get the index of the latest entry for each ALIEN_ID
latest_indices = db_TC8.loc[db_TC8.groupby('ALIEN_ID')['CREATED_TIMESTAMP'].idxmax()]

# Filter based on MASTER_FORM_TYPE
ms_type_check = latest_indices[latest_indices['MASTER_FORM_TYPE'].isin(['MT_13_EXIT'])]

status_list = ["normal" if x == "MT_13_EXIT" else "abnormal" for x in ms_type_check['MASTER_FORM_TYPE']]

abnormal_count = status_list.count('abnormal')
result = 'abnormal' if 'abnormal' in status_list else 'normal'

/tmp/ipykernel_47964/1060187526.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_TC8['CREATED_TIMESTAMP'] = pd.to_datetime(db_TC8['CREATED_TIMESTAMP'])


In [59]:
abnormal_count

0

In [55]:
result

'normal'

In [53]:
status_list

['normal', 'normal', 'normal', 'normal', 'normal']

In [48]:
result

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
51,2024-02-10,00010,00050,กรรมกร,13155987,MT_13_EXIT,Completed,2027-04-15 11:11:00
52,2024-02-10,00010,00051,กรรมกร,13155987,MT_13_EXIT,Completed,2027-04-15 11:11:00
53,2024-02-10,00010,00052,กรรมกร,13155987,MT_13_EXIT,Completed,2027-04-15 11:11:00
54,2024-02-10,00010,00053,กรรมกร,13155987,MT_13_EXIT,Completed,2027-04-15 11:11:00
55,2024-02-10,00010,00054,กรรมกร,13155987,MT_13_EXIT,Completed,2027-04-15 11:11:00


In [ ]:
result[""]

In [33]:
db_TC8_group

In [ ]:
db_TC8_MT_13 = db_TC8[db_TC8["MASTER_FORM_TYPE"] == "MT_13_EXIT"]
db_TC8_MT_59 = data_TC8[db_TC8["MASTER_FORM_TYPE"] == "MT_59"]

In [ ]:

config_case = {
    "number": 20,
    "day": 14
    }

In [60]:
#Test ID 7 **remark similar test ID 1 instead
def check_status_resign_a(data, db):
    """
    Check aliens has not yet reported their departure from the old company but has already applied for the new one.

    Parameters:
    - data (pd.DataFrame): The input data containing job information.
    - db (pd.DataFrame): The database containing status information.

    Returns:
    - str: 'normal' if the required status is present, 'abnormal' otherwise.
    """
    # Filter data based on master_form_type
    data_case = data[data['MASTER_FORM_TYPE'] == 'MT_59']
    
    # Merge data and db on 'ALIEN_ID'
    merged_id = pd.merge(data_case, db, on='ALIEN_ID', how='inner',suffixes=('_A', '_B'))
    
    # Check if the required status is present in the merged data
    return 'abnormal' if 'MT_13_EXIT' not in merged_id['MASTER_FORM_TYPE_B'].values else 'normal'


#Test ID 8 ***
def check_status_resign_b(db):
    """
    Check aliens has moved out of the company but has not yet reported their arrival at the new place.

    Parameters:
    - db (pd.DataFrame): The database containing status information.

    Returns:
    - str: 'normal' if the required status is not present, 'abnormal' otherwise.
    """
    # Check if the required status is present in the db
    # Ensure CREATED_TIMESTAMP is in datetime format
    db_TC8['CREATED_TIMESTAMP'] = pd.to_datetime(db_TC8['CREATED_TIMESTAMP'])

    # Get the index of the latest entry for each ALIEN_ID
    latest_indices = db_TC8.loc[db_TC8.groupby('ALIEN_ID')['CREATED_TIMESTAMP'].idxmax()]

    # Filter based on MASTER_FORM_TYPE
    ms_type_check = latest_indices[latest_indices['MASTER_FORM_TYPE'].isin(['MT_13_EXIT'])]

    status_list = ["normal" if x == "MT_13_EXIT" else "abnormal" for x in ms_type_check['MASTER_FORM_TYPE']]

    abnormal_count = status_list.count('abnormal')
    result = 'abnormal' if 'abnormal' in status_list else 'normal'
    
    return result, abnormal_count

In [61]:
result, abnormal_count = check_status_resign_b(db_TC8)

/tmp/ipykernel_47964/323955613.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_TC8['CREATED_TIMESTAMP'] = pd.to_datetime(db_TC8['CREATED_TIMESTAMP'])


In [62]:
result

'normal'

In [63]:
abnormal_count

0

In [ ]:
check_status_resign_a(data_TC8, db_TC8)

ValueError: You are trying to merge on float64 and object columns for key 'ALIEN_ID'. If you wish to proceed you should use pd.concat

TC14

In [14]:
config_case = {
    "number": 20,
    "day": 15
    }

In [15]:
data_TC14 = data[data["FORM_ID"].isin(["00019","00020"])]
db_TC14 = db[db["FORM_ID"].isin(["00019","00020"])]

In [16]:
data_case = data_TC14.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
db_case = db_TC14.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
concat_df = pd.concat([db_case, data_case], axis=0).reset_index(drop=True)

In [17]:
concat_df

,CREATED_TIMESTAMP,EMPLOYER_NO,FORM_ID,ALIEN_COUNT
0,2024-04-14,20152921,00020,10
1,2024-04-30,20152921,00019,12
2,2024-06-30,18152918,00019,12
3,2024-07-14,18152918,00020,10


In [18]:
anomaly = check_scenario(concat_df, config_case['number'], config_case['day'])
anomaly

2024-04-15 00:00:00
  CREATED_TIMESTAMP EMPLOYER_NO FORM_ID  ALIEN_COUNT
3        2024-07-14    18152918   00020           10
2        2024-06-30    18152918   00019           12
1        2024-04-30    20152921   00019           12
34
  CREATED_TIMESTAMP EMPLOYER_NO FORM_ID  ALIEN_COUNT
3        2024-07-14    18152918   00020           10
2        2024-06-30    18152918   00019           12
22


'ผิดปกติ: Count exceeds limit but days are within limit'

In [ ]:
check_relocate_condition_from_A_to_B(data_case, db_case, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B)

In [397]:
def check_expire_condition(data, db):
    """
    remaining period between the application submission date and the expiration date of the work permit is less than or equal to 30 days.

    Parameters:
    - data (pd.DataFrame): The input data containing job information.
    - db (pd.DataFrame): The database containing exit information.

    Returns:
    - str: 'normal' if the timestamp condition is met, 'abnormal' otherwise.
    """
    # Filter data based on master_form_type
    data_case = data[data['MASTER_FORM_TYPE'] == 'MT_59']
    db_case = db[db['MASTER_FORM_TYPE'] == 'MT_13_EXIT']
    
    # Merge data and db on 'ALIEN_ID'
    merged_id = pd.merge(data_case, db_case, on='ALIEN_ID', how='inner')
    print(merged_id)
    
    # Convert timestamps to datetime
    merged_id['CREATED_TIMESTAMP_x'] = pd.to_datetime(merged_id['CREATED_TIMESTAMP_x'])
    merged_id['VALID_UNTIL_y'] = pd.to_datetime(merged_id['VALID_UNTIL_y'])
    
    # Calculate the condition
    merged_id['is_abnormal'] = (merged_id['CREATED_TIMESTAMP_x'] + pd.Timedelta(days=30)) > merged_id['VALID_UNTIL_y']
    print(merged_id)
    
    aliens_abnormal_list = merged_id[merged_id['is_abnormal'] == True]["ALIEN_ID"]
    count_abnormal = len(aliens_abnormal_list)
    data_abnormal = data[data["ALIEN_ID"].isin(aliens_abnormal_list)]
    data_abnormal["status"] = "abnormal"
    data_abnormal["abnormal_code"] = "TC02"
    data_abnormal["abnormal_desc"] = "The application submission date and the expiration date of the work permit is less than or equal to 30 days."
    
    # Determine anomaly status
    result = 'normal' if (len(merged_id["is_abnormal"]) - count_abnormal) > 0 else 'abnormal'
    return {"result":result, "count_abnormal": count_abnormal, "data": data_abnormal} 

In [398]:
data_TC6 = data[data["FORM_ID"].isin(["00006","00009"])]
db_TC6 = db[db["FORM_ID"].isin(["00006","00009"])]

In [399]:
result = check_expire_condition(data_TC6, db_TC6)

  CREATED_TIMESTAMP_x FORM_ID_x ALIEN_ID JOB_DESCRIPTION_x EMPLOYER_NO_x  \
0          2024-06-27     00006    00501            กรรมกร      12346801   
1          2024-06-27     00009    00045            กรรมกร      12346803   
2          2024-06-27     00009    00046            กรรมกร      12346803   
3          2024-06-27     00009    00047            กรรมกร      12346803   
4          2024-06-27     00009    00048            กรรมกร      12346803   
5          2024-06-27     00009    00049            กรรมกร      12346803   

  MASTER_FORM_TYPE_x     MASTER_FORM_STATUS_x       VALID_UNTIL_x  \
0              MT_59  Consider request round1 2024-07-15 11:11:00   
1              MT_59  Consider request round1 2027-04-15 11:11:00   
2              MT_59  Consider request round1 2027-04-15 11:11:00   
3              MT_59  Consider request round1 2027-04-15 11:11:00   
4              MT_59  Consider request round1 2027-04-15 11:11:00   
5              MT_59  Consider request round1 2027-04

/tmp/ipykernel_47964/1950830304.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_abnormal["status"] = "abnormal"
/tmp/ipykernel_47964/1950830304.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_abnormal["abnormal_code"] = "TC02"
/tmp/ipykernel_47964/1950830304.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [400]:
result

{'result': 'normal',
 'count_abnormal': 1,
 'data':    CREATED_TIMESTAMP FORM_ID ALIEN_ID JOB_DESCRIPTION EMPLOYER_NO  \
 41        2024-06-27   00006    00501          กรรมกร    12346801   
 
    MASTER_FORM_TYPE       MASTER_FORM_STATUS         VALID_UNTIL    status  \
 41            MT_59  Consider request round1 2024-07-15 11:11:00  abnormal   
 
    abnormal_code                                      abnormal_desc  
 41          TC02  The application submission date and the expira...  }

In [401]:
result["data"]

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,status,abnormal_code,abnormal_desc
41,2024-06-27,00006,00501,กรรมกร,12346801,MT_59,Consider request round1,2024-07-15 11:11:00,abnormal,TC02,The application submission date and the expira...


In [11]:
def flow_2(data, db, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B):
    """
    Flow 2: Check various conditions for aliens moving from employer A to employer B.

    Parameters:
    - data (pd.DataFrame): Current data.
    - db (pd.DataFrame): Historical data.
    - config_case (dict): Configuration case containing 'number' and 'day' keys.
    - EMPLOYER_NO_A (str): Employer number for location A.
    - EMPLOYER_NO_B (str): Employer number for location B.

    Returns:
    - dict: A dictionary containing the status and message of the check.
    """
    # Define messages
    messages = {
        "normal": "Aliens moved from A to B not exceeding the limit of people and have been relocated for less than a specified number of days.",
        "abnormal_relocate": "Aliens moved from A to B exceeding the limit of people and have been relocated for more than a specified number of days.",
        "abnormal_resign_B": "Aliens has moved out of the company but has not yet reported.",
        "abnormal_resign_A": "Aliens has not yet reported their departure from the old company but has already applied for the new one.",
        "abnormal_expire": "The application submission date and the expiration date of the work permit is less than or equal to 30 days."
    }

    # Check conditions in sequence
    result_check_inform_exit = check_inform_exit(data, db)
    if result_check_inform_exit["result"] == "normal": 
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        data_case = data[~data['ALIEN_ID'].isin(alien_ids_to_drop)]
        db_case = db[~db['ALIEN_ID'].isin(alien_ids_to_drop)]
        
        result_check_expire_condition = check_expire_condition(data_case, db_case) 
        
        if result_check_expire_condition["result"] == "normal":
            alien_ids_to_drop_2 = result_check_expire_condition["data"]['ALIEN_ID'].unique()
            data_case_2 = data[~data['ALIEN_ID'].isin(alien_ids_to_drop_2)]
            db_case_2 = db[~db['ALIEN_ID'].isin(alien_ids_to_drop_2)]
            
            result_check_relocate_condition_from_A_to_B = check_relocate_condition_from_A_to_B(data_case_2,
                                                                                                    db_case_2, 
                                                                                                    config_case, 
                                                                                                    EMPLOYER_NO_A, 
                                                                                                    EMPLOYER_NO_B)
            
            if result_check_relocate_condition_from_A_to_B["result"] == "normal":
                return {"status": "normal", 
                        "message": messages["normal"],
                        "count_abnormal": result_check_relocate_condition_from_A_to_B['count_abnormal'],
                        "total_relocate_day": result_check_relocate_condition_from_A_to_B['total_relocate_day'],
                        "data": result_check_relocate_condition_from_A_to_B["data"]
                        }
            else:
                return {"status": "abnormal", 
                        "message": messages["abnormal_relocate"],
                        "count_abnormal": result_check_relocate_condition_from_A_to_B['count_abnormal'],
                        "total_relocate_day": result_check_relocate_condition_from_A_to_B['total_relocate_day'],
                        "data": result_check_relocate_condition_from_A_to_B["data"]
                        }
        else:
            return {"status": "abnormal", 
                    "message": messages["abnormal_expire"],
                    "count_abnormal": result_check_expire_condition['count_abnormal'],
                    "data": result_check_expire_condition["data"]
                    }
    else:
        return {
            "status": "abnormal",
            "message": messages["abnormal_resign_A"],
            "count_abnormal": result_check_inform_exit['count_abnormal'],
            "data": result_check_inform_exit["data"]
        }

In [19]:
data_TC7 = data[data["FORM_ID"].isin(["00019","00020"])]
db_TC7 = db[db["FORM_ID"].isin(["00019","00020"])]

config_case = {
    "number": 20,
    "day": 14
    }

In [20]:
result = flow_2(data_TC7, db_TC7, config_case, db_TC7["EMPLOYER_NO"].to_list()[0], data_TC7["EMPLOYER_NO"].to_list()[0])

In [21]:
result

{'status': 'abnormal',
 'message': 'Aliens moved from A to B exceeding the limit of people and have been relocated for more than a specified number of days.',
 'count_abnormal': 22,
 'total_relocate_day': 14,
 'data':     CREATED_TIMESTAMP FORM_ID ALIEN_ID    JOB_DESCRIPTION EMPLOYER_NO  \
 121        2024-07-01   00019    00120             กรรมกร    18152918   
 122        2024-07-01   00019    00121             กรรมกร    18152918   
 123        2024-07-01   00019    00122             กรรมกร    18152918   
 124        2024-07-01   00019    00123             กรรมกร    18152918   
 125        2024-07-01   00019    00124             กรรมกร    18152918   
 126        2024-07-01   00019    00125             กรรมกร    18152918   
 127        2024-07-01   00019    00126             กรรมกร    18152918   
 128        2024-07-01   00019    00127             กรรมกร    18152918   
 129        2024-07-01   00019    00128             กรรมกร    18152918   
 130        2024-07-01   00019    00129    

In [22]:
result["data"]

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,status,abnormal_code,abnormal_desc
121,2024-07-01,00019,00120,กรรมกร,18152918,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,Aliens moved to B exceeding the limit of peopl...
122,2024-07-01,00019,00121,กรรมกร,18152918,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,Aliens moved to B exceeding the limit of peopl...
123,2024-07-01,00019,00122,กรรมกร,18152918,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,Aliens moved to B exceeding the limit of peopl...
124,2024-07-01,00019,00123,กรรมกร,18152918,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,Aliens moved to B exceeding the limit of peopl...
125,2024-07-01,00019,00124,กรรมกร,18152918,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,Aliens moved to B exceeding the limit of peopl...
126,2024-07-01,00019,00125,กรรมกร,18152918,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,Aliens moved to B exceeding the limit of peopl...
127,2024-07-01,00019,00126,กรรมกร,18152918,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,Aliens moved to B exceeding the limit of peopl...
128,2024-07-01,00019,00127,กรรมกร,18152918,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,Aliens moved to B exceeding the limit of peopl...
129,2024-07-01,00019,00128,กรรมกร,18152918,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,Aliens moved to B exceeding the limit of peopl...
130,2024-07-01,00019,00129,กรรมกร,18152918,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,Aliens moved to B exceeding the limit of peopl...


In [7]:
from flow.module import *
def check_relocate_condition_from_A_to_B(data, db, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B):
    """
    Check if a group of aliens moved from A to B exceeding the limit of people and have been relocated for more than a specified number of days.

    Parameters:
    - data (pd.DataFrame): DataFrame containing access preparation data.
    - db (pd.DataFrame): DataFrame containing exit preparation data.
    - config_case (dict): Configuration case containing 'number' and 'day' keys.
    - EMPLOYER_NO_A (str): Employer number for location A.
    - EMPLOYER_NO_B (str): Employer number for location B.

    Returns:
    - tuple: A tuple containing the anomaly status and the merged DataFrame.
    """
    # Filter for MT_13_EXIT at location A
    db_exit_filter = db[(db["EMPLOYER_NO"] == EMPLOYER_NO_A) & (db["MASTER_FORM_TYPE"] == "MT_13_EXIT")]

    # Filter for MT_59 at location B
    db_access_filter = data[(data["EMPLOYER_NO"] == EMPLOYER_NO_B) & (data["MASTER_FORM_TYPE"] == "MT_59")]

    # Merge on ALIEN_ID
    merged_alien_id = pd.merge(db_exit_filter, db_access_filter, on='ALIEN_ID', how='inner', suffixes=('_A', '_B'))

    # Group by date and employer numbers
    count_alien_date = merged_alien_id.groupby(['CREATED_TIMESTAMP_A', "EMPLOYER_NO_A", "EMPLOYER_NO_B"]).agg(
        ALIEN_COUNT=('ALIEN_ID', 'count')
    ).reset_index().rename(columns={'CREATED_TIMESTAMP_A': 'CREATED_TIMESTAMP'})
        
    # Check relocation condition
    # Check relocation condition
    result = check_relocate_condition_from_B(data, count_alien_date, config_case['number'], config_case['day'])
    # result = check_relocate_condition_from_B(
    #     data,
    #     count_alien_date, 
    #     config_case['number'], 
    #     config_case['day']
    # )
    return result

In [8]:
check_relocate_condition_from_B?

Signature: check_relocate_condition_from_B(data, df, limit_count, limit_days)
Docstring:
Check if a group of aliens moved to location B within the limit of people and have been relocated for more than a specified number of days. 

Parameters:
- df (pd.DataFrame): DataFrame containing relocation data with 'CREATED_TIMESTAMP' and 'ALIEN_COUNT' columns.
- end_date (str or pd.Timestamp): The end date for the check in 'YYYY-MM-DD' format or as a Timestamp.
- limit_count (int): The limit of people allowed to relocate.
- limit_days (int): The number of days to check for relocation.

Returns:
- str: A message indicating whether the condition is normal or abnormal.
File:      /opt/project/src/DOE/src/function/flow/module.py
Type:      function

In [10]:
result = check_relocate_condition_from_A_to_B(data_TC7, db_TC7, config_case, db_TC7["EMPLOYER_NO"].to_list()[0], data_TC7["EMPLOYER_NO"].to_list()[0])

/opt/project/src/DOE/src/function/flow/module.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["status"] = "normal"
/opt/project/src/DOE/src/function/flow/module.py:241: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["abnormal_code"] = "pass"
/opt/project/src/DOE/src/function/flow/module.py:242: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [45]:
from flow.module import *
def flow_2(data, db, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B):
    """
    Flow 2: Check various conditions for aliens moving from employer A to employer B.

    Parameters:
    - data (pd.DataFrame): Current data.
    - db (pd.DataFrame): Historical data.
    - config_case (dict): Configuration case containing 'number' and 'day' keys.
    - EMPLOYER_NO_A (str): Employer number for location A.
    - EMPLOYER_NO_B (str): Employer number for location B.

    Returns:
    - dict: A dictionary containing the status and message of the check, along with all relevant data.
    """
    # Define messages
    messages = {
        "normal": "Aliens moved from A to B not exceeding the limit of people and have been relocated for less than a specified number of days.",
        "abnormal_relocate": "Aliens moved from A to B exceeding the limit of people and have been relocated for more than a specified number of days.",
        "abnormal_resign_B": "Aliens has moved out of the company but has not yet reported.",
        "abnormal_resign_A": "Aliens has not yet reported their departure from the old company but has already applied for the new one.",
        "abnormal_expire": "The application submission date and the expiration date of the work permit is less than or equal to 30 days."
    }

    # Check conditions in sequence
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'total_relocate_day': 0,
        'data': None
    }
    
    if result_check_inform_exit["result"] == "normal": 
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        data_case = data[~data['ALIEN_ID'].isin(alien_ids_to_drop)]
        db_case = db[~db['ALIEN_ID'].isin(alien_ids_to_drop)]
        
        result_check_expire_condition = check_expire_condition(data_case, db_case) 
        
        if result_check_expire_condition["result"] == "normal":
            alien_ids_to_drop_2 = result_check_expire_condition["data"]['ALIEN_ID'].unique()
            data_case_2 = data_case[~data_case['ALIEN_ID'].isin(alien_ids_to_drop_2)]
            db_case_2 = db_case[~db_case['ALIEN_ID'].isin(alien_ids_to_drop_2)]
            #print(data_case_2)
            
            result_check_relocate_condition_from_A_to_B = check_relocate_condition_from_A_to_B(data_case_2,
                                                                                                    db_case_2, 
                                                                                                    config_case, 
                                                                                                    EMPLOYER_NO_A, 
                                                                                                    EMPLOYER_NO_B)
            
            if result_check_relocate_condition_from_A_to_B["result"] == "normal":
                result.update({
                    "status": "normal", 
                    "message": messages["normal"],
                    "count_abnormal": result_check_relocate_condition_from_A_to_B['count_abnormal'],
                    "total_relocate_day": result_check_relocate_condition_from_A_to_B['total_relocate_day'],
                    "data": pd.concat([result_check_inform_exit["data"], result_check_relocate_condition_from_A_to_B["data"]], ignore_index=True)
                })
            else:
                result.update({
                    "status": "abnormal", 
                    "message": messages["abnormal_relocate"],
                    "count_abnormal": result_check_relocate_condition_from_A_to_B['count_abnormal'],
                    "total_relocate_day": result_check_relocate_condition_from_A_to_B['total_relocate_day'],
                    "data": pd.concat([result_check_inform_exit["data"], result_check_relocate_condition_from_A_to_B["data"]], ignore_index=True)
                })
        else:
            result.update({
                "status": "abnormal", 
                "message": messages["abnormal_expire"],
                "count_abnormal": result_check_expire_condition['count_abnormal'],
                "data": result_check_expire_condition["data"]
            })
    else:
        result.update({
            "status": "abnormal",
            "message": messages["abnormal_resign_A"],
            "count_abnormal": result_check_inform_exit['count_abnormal'],
            "data": result_check_inform_exit["data"]
        })
    
    return result


In [70]:
from flow.module import *

def flow_2(data, db, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B):
    """
    Flow 2: Check various conditions for aliens moving from employer A to employer B.

    Parameters:
    - data (pd.DataFrame): Current data.
    - db (pd.DataFrame): Historical data.
    - config_case (dict): Configuration case containing 'number' and 'day' keys.
    - EMPLOYER_NO_A (str): Employer number for location A.
    - EMPLOYER_NO_B (str): Employer number for location B.

    Returns:
    - dict: A dictionary containing the status and message of the check, along with all relevant data.
    """
    # Define messages
    messages = {
        "normal": "Aliens moved from A to B not exceeding the limit of people and have been relocated for less than a specified number of days.",
        "abnormal_relocate": "Aliens moved from A to B exceeding the limit of people and have been relocated for more than a specified number of days.",
        "abnormal_resign_B": "Aliens has moved out of the company but has not yet reported.",
        "abnormal_resign_A": "Aliens has not yet reported their departure from the old company but has already applied for the new one.",
        "abnormal_expire": "The application submission date and the expiration date of the work permit is less than or equal to 30 days."
    }

    # Check conditions in sequence
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'total_relocate_day': 0,
        'data': None
    }
    
    if result_check_inform_exit["result"] == "normal": 
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        data_case = data[~data['ALIEN_ID'].isin(alien_ids_to_drop)]
        db_case = db[~db['ALIEN_ID'].isin(alien_ids_to_drop)]
        
        result_check_expire_condition = check_expire_condition(data_case, db_case) 
        
        if result_check_expire_condition["result"] == "normal":
            alien_ids_to_drop_2 = result_check_expire_condition["data"]['ALIEN_ID'].unique()
            data_case_2 = data_case[~data_case['ALIEN_ID'].isin(alien_ids_to_drop_2)]
            db_case_2 = db_case[~db_case['ALIEN_ID'].isin(alien_ids_to_drop_2)]
            #print(data_case_2)
            
            result_check_relocate_condition_from_A_to_B = check_relocate_condition_from_A_to_B(data_case_2,
                                                                                                    db_case_2, 
                                                                                                    config_case, 
                                                                                                    EMPLOYER_NO_A, 
                                                                                                    EMPLOYER_NO_B)
            
            if result_check_relocate_condition_from_A_to_B["result"] == "normal":
                result.update({
                    "status": "normal", 
                    "message": messages["normal"],
                    "count_abnormal": result_check_relocate_condition_from_A_to_B['count_abnormal'],
                    "total_relocate_day": result_check_relocate_condition_from_A_to_B['total_relocate_day'],
                    "data": pd.concat([result_check_inform_exit["data"], result_check_relocate_condition_from_A_to_B["data"]], ignore_index=True).to_dict(orient="records")
                })
            else:
                result.update({
                    "status": "abnormal", 
                    "message": messages["abnormal_relocate"],
                    "count_abnormal": result_check_relocate_condition_from_A_to_B['count_abnormal'],
                    "total_relocate_day": result_check_relocate_condition_from_A_to_B['total_relocate_day'],
                    "data": pd.concat([result_check_inform_exit["data"], result_check_relocate_condition_from_A_to_B["data"]], ignore_index=True).to_dict(orient="records")
                })
        else:
            result.update({
                "status": "abnormal", 
                "message": messages["abnormal_expire"],
                "count_abnormal": result_check_expire_condition['count_abnormal'],
                "data": result_check_expire_condition["data"].to_dict(orient="records")
            })
    else:
        result.update({
            "status": "abnormal",
            "message": messages["abnormal_resign_A"],
            "count_abnormal": result_check_inform_exit['count_abnormal'],
            "data": result_check_inform_exit["data"].to_dict(orient="records")
        })
    
    return result


In [75]:
def flow_4(data, db, config_case):
    """
    Flow 4: Check relocation conditions and departure reporting status of aliens.

    Parameters:
    - data (pd.DataFrame): Current data.
    - db (pd.DataFrame): Historical data.
    - config_case (dict): Configuration case containing number and day limits.

    Returns:
    - dict: A dictionary containing the status, message, count of abnormalities, total relocate days, and all relevant data.
    """
    messages = {
        "normal": "Aliens moved to B not exceeding the limit of people and have been relocated for less than a specified number of days.",
        "abnormal_relocate_B": "Aliens moved to B exceeding the limit of people and have been relocated for more than a specified number of days.",
        "check_inform_exit": "Aliens have not yet reported their departure from the old company but have already applied for the new one."
    }
    
    # Check if aliens have reported their departure
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'total_relocate_day': 0,
        'data': None
    }
    
    if result_check_inform_exit["result"] == "normal":
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        data_case = data[~data['ALIEN_ID'].isin(alien_ids_to_drop)]
        db_case = db[~db['ALIEN_ID'].isin(alien_ids_to_drop)]
        
        # Create concat_df
        data_case_agg = data_case.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
        db_case_agg = db_case.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
        concat_df = pd.concat([db_case_agg, data_case_agg], axis=0).reset_index(drop=True)
        
        result_check_relocate_condition_from_B = check_relocate_condition_from_B(data_case, concat_df, config_case["number"], config_case["day"])
        
        if result_check_relocate_condition_from_B["result"] == "normal":
            result.update({
                "status": "normal", 
                "message": messages["normal"], 
                "count_abnormal": result_check_relocate_condition_from_B['count_abnormal'],
                "total_relocate_day": result_check_relocate_condition_from_B['total_relocate_day'],
                "data": pd.concat([result_check_inform_exit['data'], result_check_relocate_condition_from_B["data"]], ignore_index=True).to_dict(orient='records')
            })
        else:
            result.update({
                "status": "abnormal", 
                "message": messages["abnormal_relocate_B"],
                "count_abnormal": result_check_relocate_condition_from_B['count_abnormal'],
                "total_relocate_day": result_check_relocate_condition_from_B['total_relocate_day'],
                "data": pd.concat([result_check_inform_exit['data'], result_check_relocate_condition_from_B["data"]], ignore_index=True).to_dict(orient='records')
            })
    
    else:
        result.update({
            "status": "abnormal", 
            "message": messages["check_inform_exit"],
            "count_abnormal": result_check_inform_exit['count_abnormal'],
            "data": result_check_inform_exit['data'].to_dict(orient='records')
        })
    
    return result


In [82]:
data_TC7 = data[data["FORM_ID"].isin(["00022","00023","00024"])]
db_TC7 = db[db["FORM_ID"].isin(["00022","00023","00024"])]

config_case = {
    "number": 20,
    "day": 14
    }

In [83]:
data_TC7

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
144,2024-06-27,00022,00142,กรรมกร,20152921,MT_59,Consider request round1,2027-04-17 11:10:59.995
145,2024-06-27,00022,00143,กรรมกร,20152921,MT_59,Consider request round1,2027-04-17 11:10:59.995
146,2024-06-27,00022,00144,กรรมกร,20152921,MT_59,Consider request round1,2027-04-17 11:10:59.995
147,2024-06-27,00022,00145,กรรมกร,20152921,MT_59,Consider request round1,2027-04-17 11:10:59.995
148,2024-06-27,00022,00146,กรรมกร,20152921,MT_59,Consider request round1,2027-04-17 11:10:59.995
149,2024-06-27,00022,00147,กรรมกร,20152921,MT_59,Consider request round1,2027-04-17 11:10:59.995
150,2024-06-27,00022,00148,กรรมกร,20152921,MT_59,Consider request round1,2027-04-17 11:10:59.995
151,2024-06-27,00022,00149,กรรมกร,20152921,MT_59,Consider request round1,2027-04-17 11:10:59.995
152,2024-06-27,00022,00150,กรรมกร,20152921,MT_59,Consider request round1,2027-04-17 11:10:59.995
153,2024-06-27,00022,00151,กรรมกร,20152921,MT_59,Consider request round1,2027-04-17 11:10:59.995


In [84]:
config_case = {
    "number": 50,
    "day": 20
    }
result = flow_4(data_TC7, db_TC7, config_case)

In [86]:
result

{'status': 'normal',
 'message': 'Aliens moved to B not exceeding the limit of people and have been relocated for less than a specified number of days.',
 'count_abnormal': 0,
 'total_relocate_day': 'pass',
 'data': [{'CREATED_TIMESTAMP': Timestamp('2024-06-27 00:00:00'),
   'FORM_ID': '00022',
   'ALIEN_ID': '00142',
   'JOB_DESCRIPTION': 'กรรมกร',
   'EMPLOYER_NO': '20152921',
   'MASTER_FORM_TYPE': 'MT_59',
   'MASTER_FORM_STATUS': 'Consider request round1',
   'VALID_UNTIL': Timestamp('2027-04-17 11:10:59.995000'),
   'status': 'normal',
   'abnormal_code': 'pass',
   'abnormal_desc': 'pass'},
  {'CREATED_TIMESTAMP': Timestamp('2024-06-27 00:00:00'),
   'FORM_ID': '00022',
   'ALIEN_ID': '00143',
   'JOB_DESCRIPTION': 'กรรมกร',
   'EMPLOYER_NO': '20152921',
   'MASTER_FORM_TYPE': 'MT_59',
   'MASTER_FORM_STATUS': 'Consider request round1',
   'VALID_UNTIL': Timestamp('2027-04-17 11:10:59.995000'),
   'status': 'normal',
   'abnormal_code': 'pass',
   'abnormal_desc': 'pass'},
  {'C

In [85]:
result["data"]

[{'CREATED_TIMESTAMP': Timestamp('2024-06-27 00:00:00'),
  'FORM_ID': '00022',
  'ALIEN_ID': '00142',
  'JOB_DESCRIPTION': 'กรรมกร',
  'EMPLOYER_NO': '20152921',
  'MASTER_FORM_TYPE': 'MT_59',
  'MASTER_FORM_STATUS': 'Consider request round1',
  'VALID_UNTIL': Timestamp('2027-04-17 11:10:59.995000'),
  'status': 'normal',
  'abnormal_code': 'pass',
  'abnormal_desc': 'pass'},
 {'CREATED_TIMESTAMP': Timestamp('2024-06-27 00:00:00'),
  'FORM_ID': '00022',
  'ALIEN_ID': '00143',
  'JOB_DESCRIPTION': 'กรรมกร',
  'EMPLOYER_NO': '20152921',
  'MASTER_FORM_TYPE': 'MT_59',
  'MASTER_FORM_STATUS': 'Consider request round1',
  'VALID_UNTIL': Timestamp('2027-04-17 11:10:59.995000'),
  'status': 'normal',
  'abnormal_code': 'pass',
  'abnormal_desc': 'pass'},
 {'CREATED_TIMESTAMP': Timestamp('2024-06-27 00:00:00'),
  'FORM_ID': '00022',
  'ALIEN_ID': '00144',
  'JOB_DESCRIPTION': 'กรรมกร',
  'EMPLOYER_NO': '20152921',
  'MASTER_FORM_TYPE': 'MT_59',
  'MASTER_FORM_STATUS': 'Consider request round1'

In [ ]:
# Initialize counters and lists
count_abnormal = 0
job_abnormal_list = []

# Extract specific jobs from config_case excluding "อื่นๆ"
specific_jobs = [job_config['job'] for job_config in config_case if job_config['job'] != "อื่นๆ"]

# Check each job configuration
for job_config in config_case:
    job = job_config['job']
    expected_number = job_config['number']
    
    # Calculate the total count of aliens for the specified job
    if job == "อื่นๆ":
        # Calculate the sum of ALIEN_COUNT for jobs not listed in specific jobs
        job_count = data_case[~data_case['JOB_DESCRIPTION'].isin(specific_jobs)]['ALIEN_COUNT'].sum()
    else:
        job_count = data_case[data_case['JOB_DESCRIPTION'] == job]['ALIEN_COUNT'].sum()
    
    # Print job details (optional)
    # print(f'job: {job}, expected: {expected_number}, job_count: {job_count}')
    
    # If the job count exceeds the expected number, mark as abnormal
    if job_count > expected_number:
        count_abnormal += 1
        job_abnormal_list.append(job)

# Display results
print(f'Number of abnormal jobs: {count_abnormal}')
print(f'List of abnormal jobs: {job_abnormal_list}')

In [193]:
def check_job_limits(data, config_case):
    """
    Check an employer hires more than 10 aliens for different types of work and positions.

    Parameters:
    - data (pd.DataFrame): The input data containing job information.
    - config_case (list of dict): A list of dictionaries containing job configurations with 'job' and 'number' keys.

    Returns:
    - str: 'normal' if all job counts are within limits, 'abnormal' otherwise.
    """
    
    abnormal_message =  "The employer hires more than 10 aliens for different types of work and positions."       
    abnormal_code = "TC02"
    # Filter data based on form_id and master_form_type
    data_case = data[data['MASTER_FORM_TYPE'] == 'MT_59']
    
    # Group by relevant columns and count the number of aliens per job
    # data_case = data_case.groupby(['EMPLOYER_NO', 'FORM_ID', 'JOB_DESCRIPTION']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
    
    count_abnormal = 0
    job_abnormal_list = []
    
    all_jobs = [job_config['job'] for job_config in config_case]
    specific_jobs = [job_config['job'] for job_config in config_case if job_config['job'] != "N/A"]
    na_jobs = data_case[~data_case['JOB_DESCRIPTION'].isin(specific_jobs)]['JOB_DESCRIPTION'].unique()
    
    # Check each job configuration
    for job_config in config_case:
        job = job_config['job']
        expected_number = job_config['number']
        # Calculate the total count of aliens for the specified job
        if job == "N/A":
            # Calculate the sum of ALIEN_ID count for jobs not listed in specific_jobs
            job_count = data_case[~data_case['JOB_DESCRIPTION'].isin(specific_jobs)]['ALIEN_ID'].count()
        else:
            job_count = data_case[data_case['JOB_DESCRIPTION'] == job]['ALIEN_ID'].count()
            
        print(f'job: {job}, expected: {expected_number}, job_count: {job_count}')
        # If the job count exceeds the expected number, return 'abnormal'
        if job_count > expected_number:
            count_abnormal += 1
            job_abnormal_list.append(job)
            
    if "N/A" in job_abnormal_list:
        job_abnormal_list.extend(na_jobs)
    # print(f'Number of abnormal jobs: {count_abnormal}')
    # print(f'List of abnormal jobs: {job_abnormal_list}')
    
    data["status"] = data["JOB_DESCRIPTION"].apply(lambda x: 'normal' if not any(item in x for item in job_abnormal_list) else 'abnormal')        
    data["abnormal_code"] = data['status'].apply(lambda x: 'pass' if x == 'normal' else abnormal_code)
    data["abnormal_desc"] = data['status'].apply(lambda x: 'pass' if x == 'normal' else abnormal_message)
    
    
    # If all job counts are within limits, return 'normal'
    result = 'abnormal' if "normal" not in data["status"].to_list() else 'normal'
    count_abnormal = data[data["status"] == "abnormal"].shape[0]
    return {'result': result,'count_abnormal': count_abnormal, 'job_abnormal': job_abnormal_list, 'data': data}

In [194]:
data_TC7 = data[data["FORM_ID"].isin(["00003"])]
db_TC7 = db[db["FORM_ID"].isin(["00003"])]

config_case = [
    {"job": "กรรมกร", "number": 0},
    {"job": "งานทํารองเท้า", "number": 1},
    {"job": "งานทําหมวก", "number": 1},
    {"job": "N/A", "number": 2},
]

In [195]:
data_TC7

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
12,2024-04-27,00003,00013,งานทํารองเท้า,12345700,MT_59,Consider request round1,2027-04-15 11:11:00
13,2024-04-27,00003,00014,งานทําหมวก,12345700,MT_59,Consider request round1,2027-04-15 11:11:00
14,2024-04-27,00003,00015,อื่นๆ,12345700,MT_59,Consider request round1,2027-04-15 11:11:00
15,2024-04-27,00003,00016,กรรมกร,12345700,MT_59,Consider request round1,2027-04-15 11:11:00
16,2024-04-27,00003,00017,aaa,12345700,MT_59,Consider request round1,2027-04-15 11:11:00
17,2024-04-27,00003,00018,งานทํารองเท้า,12345700,MT_59,Consider request round1,2027-04-15 11:11:00
18,2024-04-27,00003,00019,งานทําหมวก,12345700,MT_59,Consider request round1,2027-04-15 11:11:00
19,2024-04-27,00003,00020,งานทําหมวก,12345700,MT_59,Consider request round1,2027-04-15 11:11:00
20,2024-04-27,00003,00021,abc,12345700,MT_59,Consider request round1,2027-04-15 11:11:00


In [196]:
result = check_job_limits(data_TC7, config_case)

['กรรมกร', 'งานทํารองเท้า', 'งานทําหมวก', 'N/A']
['อื่นๆ' 'aaa' 'abc']
['กรรมกร', 'งานทํารองเท้า', 'งานทําหมวก']
job: กรรมกร, expected: 0, job_count: 1
job: งานทํารองเท้า, expected: 1, job_count: 2
job: งานทําหมวก, expected: 1, job_count: 3
job: N/A, expected: 2, job_count: 3
Number of abnormal jobs: 4
List of abnormal jobs: ['กรรมกร', 'งานทํารองเท้า', 'งานทําหมวก', 'N/A', 'อื่นๆ', 'aaa', 'abc']


/tmp/ipykernel_22226/305426729.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["status"] = data["JOB_DESCRIPTION"].apply(lambda x: 'normal' if not any(item in x for item in job_abnormal_list) else 'abnormal')
/tmp/ipykernel_22226/305426729.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["abnormal_code"] = data['status'].apply(lambda x: 'pass' if x == 'normal' else abnormal_code)
/tmp/ipykernel_22226/305426729.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [197]:
result

{'result': 'abnormal',
 'count_abnormal': 9,
 'job_abnormal': ['กรรมกร',
  'งานทํารองเท้า',
  'งานทําหมวก',
  'N/A',
  'อื่นๆ',
  'aaa',
  'abc'],
 'data':    CREATED_TIMESTAMP FORM_ID ALIEN_ID JOB_DESCRIPTION EMPLOYER_NO  \
 12        2024-04-27   00003    00013   งานทํารองเท้า    12345700   
 13        2024-04-27   00003    00014      งานทําหมวก    12345700   
 14        2024-04-27   00003    00015           อื่นๆ    12345700   
 15        2024-04-27   00003    00016          กรรมกร    12345700   
 16        2024-04-27   00003    00017             aaa    12345700   
 17        2024-04-27   00003    00018   งานทํารองเท้า    12345700   
 18        2024-04-27   00003    00019      งานทําหมวก    12345700   
 19        2024-04-27   00003    00020      งานทําหมวก    12345700   
 20        2024-04-27   00003    00021             abc    12345700   
 
    MASTER_FORM_TYPE       MASTER_FORM_STATUS         VALID_UNTIL    status  \
 12            MT_59  Consider request round1 2027-04-15 11:11:0

In [198]:
result["data"]

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,status,abnormal_code,abnormal_desc
12,2024-04-27,00003,00013,งานทํารองเท้า,12345700,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
13,2024-04-27,00003,00014,งานทําหมวก,12345700,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
14,2024-04-27,00003,00015,อื่นๆ,12345700,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
15,2024-04-27,00003,00016,กรรมกร,12345700,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
16,2024-04-27,00003,00017,aaa,12345700,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
17,2024-04-27,00003,00018,งานทํารองเท้า,12345700,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
18,2024-04-27,00003,00019,งานทําหมวก,12345700,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
19,2024-04-27,00003,00020,งานทําหมวก,12345700,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...
20,2024-04-27,00003,00021,abc,12345700,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,TC02,The employer hires more than 10 aliens for dif...


In [97]:
from flow.module import *


def flow_1(data, db, config_case):
    """
    Flow 1: Check if aliens have reported their departure and if the employer hires within the job limits.

    Parameters:
    - data (pd.DataFrame): Current data.
    - db (pd.DataFrame): Historical data.
    - config_case (list of dict): Configuration specifying job limits, where each dict has:
        - "job" (str): Job title.
        - "number" (int): Maximum allowed number for this job.
    
    Example of `config_case`:
    config_case = [
        {"job": "กรรมกร", "number": 10},
        {"job": "งานขายของหน้าร้าน", "number": 10},
        {"job": "งานทํามือ", "number": 10},
        {"job": "N/A", "number": 10}
    ]

    Returns:
    - dict: A dictionary containing:
        - "status" (str): 'normal' or 'abnormal'.
        - "message" (str): Description of the check results.
        - "count_abnormal" (int): Number of abnormalities found.
        - "job_abnormal" (list of dict, optional): Details of job limit violations.
        - "data" (list of dict): Data of aliens relevant to the check.
    """
    # Check if aliens have reported their departure
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'data': None,
        'job_abnormal': None  # Only included if job limits check is abnormal
    }
    
    if result_check_inform_exit["result"] == "normal":
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        result_check_job_limits = check_job_limits(data[~data['ALIEN_ID'].isin(alien_ids_to_drop)], config_case)
        
        if result_check_job_limits["result"] == "normal":
            result.update({
                "status": "normal",
                "message": "Aliens have reported their departure from the old company, and the employer hires not more than 10 aliens for different types of work and positions.",
                "count_abnormal": result_check_job_limits["count_abnormal"],
                'job_abnormal': result_check_job_limits["job_abnormal"]
            })
            # Combine data from both checks into a DataFrame
            result_check_job_limits["data"]["case_code"] =  result_check_job_limits["data"]['status'].apply(lambda x: 'pass' if x == 'normal' else "R1/2")
            result_check_inform_exit["data"]["case_code"] = "R1/1"
            combined_data = pd.concat([result_check_inform_exit["data"], result_check_job_limits["data"]])
            combined_data = combined_data.reset_index(drop=True)
            result['data'] = combined_data.to_dict(orient='records')  # Convert DataFrame to dict
        else:
            result.update({
                "status": "abnormal",
                "message": "Aliens have reported their departure from the old company. However, the employer hires more than 10 aliens for different types of work and positions.",
                "count_abnormal": result_check_job_limits["count_abnormal"],
                'job_abnormal': result_check_job_limits["job_abnormal"]
            })
            # Correct status in data from inform exit check
            result_check_job_limits["data"]['status'] = 'abnormal'
            result_check_job_limits["data"]["case_code"] =  result_check_job_limits["data"]['status'].apply(lambda x: 'pass' if x == 'normal' else "R1/2")
            result_check_inform_exit["data"]["case_code"] = "R1/1"

            combined_data = pd.concat([result_check_inform_exit["data"], result_check_job_limits["data"]], ignore_index=True)
            result['data'] = combined_data.to_dict(orient="records")
    
    else:
        result.update({
            "status": "abnormal",
            "message": "Some aliens have not yet reported their departure from the old company.",
            "count_abnormal": result_check_inform_exit['count_abnormal'],
            'job_abnormal': None
        })
        result_check_inform_exit["data"]["case_code"] = "R1/1"
        result['data'] = result_check_inform_exit["data"].to_dict(orient='records')  # Convert DataFrame to dict
    
    return result

In [98]:
data_TC7 = data[data["FORM_ID"].isin(["00001","00003"])]
db_TC7 = db[db["FORM_ID"].isin(["00001","00003"])]

config_case = [
    {"job": "กรรมกร", "number": 10},
    {"job": "งานทํารองเท้า", "number": 10},
    {"job": "งานทําหมวก", "number": 10},
    {"job": "N/A", "number": 2},
]

In [99]:
result = flow_1(data_TC7, db_TC7, config_case)

/opt/project/src/DOE/src/function/flow/module.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered_abnormal['abnormal_desc'] = abnormal_message
/opt/project/src/DOE/src/function/flow/module.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["status"] = data["JOB_DESCRIPTION"].apply(lambda x: 'normal' if not any(item in x for item in job_abnormal_list) else 'abnormal')
/opt/project/src/DOE/src/function/flow/module.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [100]:
result["data"]

[{'CREATED_TIMESTAMP': Timestamp('2024-04-27 00:00:00'),
  'FORM_ID': '00001',
  'ALIEN_ID': '00001',
  'JOB_DESCRIPTION': 'กรรมกร',
  'EMPLOYER_NO': '12345678',
  'MASTER_FORM_TYPE': 'MT_59',
  'MASTER_FORM_STATUS': 'Consider request round1',
  'VALID_UNTIL': Timestamp('2027-04-14 11:10:59.995000'),
  'status': 'abnormal',
  'abnormal_desc': 'Aliens have not yet reported their departure from the old company.',
  'case_code': 'R1/1'},
 {'CREATED_TIMESTAMP': Timestamp('2024-04-27 00:00:00'),
  'FORM_ID': '00003',
  'ALIEN_ID': '00013',
  'JOB_DESCRIPTION': 'งานทํารองเท้า',
  'EMPLOYER_NO': '12345700',
  'MASTER_FORM_TYPE': 'MT_59',
  'MASTER_FORM_STATUS': 'Consider request round1',
  'VALID_UNTIL': Timestamp('2027-04-15 11:11:00'),
  'status': 'normal',
  'abnormal_desc': 'pass',
  'case_code': 'pass'},
 {'CREATED_TIMESTAMP': Timestamp('2024-04-27 00:00:00'),
  'FORM_ID': '00003',
  'ALIEN_ID': '00014',
  'JOB_DESCRIPTION': 'งานทําหมวก',
  'EMPLOYER_NO': '12345700',
  'MASTER_FORM_TYPE'

In [101]:
df_test = pd.DataFrame(result["data"])

In [102]:
df_test

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,status,abnormal_desc,case_code
0,2024-04-27,00001,00001,กรรมกร,12345678,MT_59,Consider request round1,2027-04-14 11:10:59.995,abnormal,Aliens have not yet reported their departure f...,R1/1
1,2024-04-27,00003,00013,งานทํารองเท้า,12345700,MT_59,Consider request round1,2027-04-15 11:11:00.000,normal,pass,pass
2,2024-04-27,00003,00014,งานทําหมวก,12345700,MT_59,Consider request round1,2027-04-15 11:11:00.000,normal,pass,pass
3,2024-04-27,00003,00015,อื่นๆ,12345700,MT_59,Consider request round1,2027-04-15 11:11:00.000,abnormal,The employer hires more than 10 aliens for dif...,R1/2
4,2024-04-27,00003,00016,กรรมกร,12345700,MT_59,Consider request round1,2027-04-15 11:11:00.000,normal,pass,pass
5,2024-04-27,00003,00017,aaa,12345700,MT_59,Consider request round1,2027-04-15 11:11:00.000,abnormal,The employer hires more than 10 aliens for dif...,R1/2
6,2024-04-27,00003,00018,งานทํารองเท้า,12345700,MT_59,Consider request round1,2027-04-15 11:11:00.000,normal,pass,pass
7,2024-04-27,00003,00019,งานทําหมวก,12345700,MT_59,Consider request round1,2027-04-15 11:11:00.000,normal,pass,pass
8,2024-04-27,00003,00020,งานทําหมวก,12345700,MT_59,Consider request round1,2027-04-15 11:11:00.000,normal,pass,pass
9,2024-04-27,00003,00021,abc,12345700,MT_59,Consider request round1,2027-04-15 11:11:00.000,abnormal,The employer hires more than 10 aliens for dif...,R1/2


In [17]:
from flow.module import *

def flow_2(data, db, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B):
    """
    Flow 2: Validate the movement of aliens from employer A to employer B based on various conditions.

    Parameters:
    - data (pd.DataFrame): DataFrame containing current alien movement data.
    - db (pd.DataFrame): DataFrame containing historical alien movement data.
    - config_case (dict): Dictionary containing 'number' and 'day' for configuration.
        - "number" (int): Maximum allowed number for aliens.
        - "day" (str): limit number move from A to B
        Example:
            config_case = {
                "number": 20,
                "day": 14
            }
    - EMPLOYER_NO_A (str): Employer number for location A.
    - EMPLOYER_NO_B (str): Employer number for location B.

    Returns:
    - dict: A dictionary with the following keys:
        - 'status' (str): Overall status of the check ('normal' or 'abnormal').
        - 'message' (str): Descriptive message about the check result.
        - 'count_abnormal' (int): Count of abnormal records.
        - 'total_relocate_day' (int): Total days aliens have been relocated.
        - 'data' (list of dict): Relevant data records.
    """
    # Define messages
    messages = {
        "normal": "Aliens moved from A to B not exceeding the limit of people and have been relocated for less than a specified number of days.",
        "abnormal_relocate": "Aliens moved from A to B exceeding the limit of people and have been relocated for more than a specified number of days.",
        "abnormal_resign_B": "Aliens has moved out of the company but has not yet reported.",
        "abnormal_resign_A": "Aliens has not yet reported their departure from the old company but has already applied for the new one.",
        "abnormal_expire": "The application submission date and the expiration date of the work permit is less than or equal to 30 days."
    }

    # Check conditions in sequence
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'total_relocate_day': 0,
        'data': None
    }
    
    if result_check_inform_exit["result"] == "normal": 
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        data_case = data[~data['ALIEN_ID'].isin(alien_ids_to_drop)]
        db_case = db[~db['ALIEN_ID'].isin(alien_ids_to_drop)]
        
        result_check_expire_condition = check_expire_condition(data_case, db_case) 
        
        if result_check_expire_condition["result"] == "normal":
            alien_ids_to_drop_2 = result_check_expire_condition["data"]['ALIEN_ID'].unique()
            data_case_2 = data_case[~data_case['ALIEN_ID'].isin(alien_ids_to_drop_2)]
            db_case_2 = db_case[~db_case['ALIEN_ID'].isin(alien_ids_to_drop_2)]
            
            result_check_relocate_condition_from_A_to_B = check_relocate_condition_from_A_to_B(data_case_2,
                                                                                                    db_case_2, 
                                                                                                    config_case, 
                                                                                                    EMPLOYER_NO_A, 
                                                                                                    EMPLOYER_NO_B)
            
            if result_check_relocate_condition_from_A_to_B["result"] == "normal":
                result.update({
                    "status": "normal", 
                    "message": messages["normal"],
                    "count_abnormal": result_check_relocate_condition_from_A_to_B['count_abnormal'],
                    "total_relocate_day": result_check_relocate_condition_from_A_to_B['total_relocate_day'],
                    "data": pd.concat([result_check_inform_exit["data"], result_check_expire_condition["data"], result_check_relocate_condition_from_A_to_B["data"]], ignore_index=True).to_dict(orient="records")
                })
            else:
                result.update({
                    "status": "abnormal", 
                    "message": messages["abnormal_relocate"],
                    "count_abnormal": result_check_relocate_condition_from_A_to_B['count_abnormal'],
                    "total_relocate_day": result_check_relocate_condition_from_A_to_B['total_relocate_day'],
                    "data": pd.concat([result_check_inform_exit["data"], result_check_expire_condition["data"], result_check_relocate_condition_from_A_to_B["data"]], ignore_index=True).to_dict(orient="records")
                })
        else:
            result.update({
                "status": "abnormal", 
                "message": messages["abnormal_expire"],
                "count_abnormal": result_check_expire_condition['count_abnormal'],
                "total_relocate_day": None,
                "data": pd.concat([result_check_inform_exit["data"], result_check_expire_condition["data"]], ignore_index=True).to_dict(orient="records")
            })
    else:
        result.update({
            "status": "abnormal",
            "message": messages["abnormal_resign_A"],
            "count_abnormal": result_check_inform_exit['count_abnormal'],
            "total_relocate_day": None,
            "data": result_check_inform_exit["data"].to_dict(orient="records")
        })
    
    return result

In [47]:
from flow.module import *

def flow_2(data, db, config_case, EMPLOYER_NO_A, EMPLOYER_NO_B):
    """
    Flow 2: Validate the movement of aliens from employer A to employer B based on various conditions.

    Parameters:
    - data (pd.DataFrame): DataFrame containing current alien movement data.
    - db (pd.DataFrame): DataFrame containing historical alien movement data.
    - config_case (dict): Dictionary containing 'number' and 'day' for configuration.
        - "number" (int): Maximum allowed number for aliens.
        - "day" (str): limit number move from A to B
        Example:
            config_case = {
                "number": 20,
                "day": 14
            }
    - EMPLOYER_NO_A (str): Employer number for location A.
    - EMPLOYER_NO_B (str): Employer number for location B.

    Returns:
    - dict: A dictionary with the following keys:
        - 'status' (str): Overall status of the check ('normal' or 'abnormal').
        - 'message' (str): Descriptive message about the check result.
        - 'count_abnormal' (int): Count of abnormal records.
        - 'total_relocate_day' (int): Total days aliens have been relocated.
        - 'data' (list of dict): Relevant data records.
    """
    # Define messages
    messages = {
        "normal": "Aliens moved from A to B not exceeding the limit of people and have been relocated for less than a specified number of days.",
        "abnormal_relocate": "Aliens moved from A to B exceeding the limit of people and have been relocated for more than a specified number of days.",
        "abnormal_resign_B": "Aliens has moved out of the company but has not yet reported.",
        "abnormal_resign_A": "Aliens has not yet reported their departure from the old company but has already applied for the new one.",
        "abnormal_expire": "The application submission date and the expiration date of the work permit is less than or equal to 30 days."
    }

    # Check conditions in sequence
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'total_relocate_day': 0,
        'data': None
    }
    
    if result_check_inform_exit["result"] == "normal": 
        
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        data_case = data[~data['ALIEN_ID'].isin(alien_ids_to_drop)]
        db_case = db[~db['ALIEN_ID'].isin(alien_ids_to_drop)]
        
        result_check_expire_condition = check_expire_condition(data_case, db_case) 
        
        if result_check_expire_condition["result"] == "normal":
            alien_ids_to_drop_2 = result_check_expire_condition["data"]['ALIEN_ID'].unique()
            data_case_2 = data_case[~data_case['ALIEN_ID'].isin(alien_ids_to_drop_2)]
            db_case_2 = db_case[~db_case['ALIEN_ID'].isin(alien_ids_to_drop_2)]
            
            result_check_relocate_condition_from_A_to_B = check_relocate_condition_from_A_to_B(data_case_2,
                                                                                                    db_case_2, 
                                                                                                    config_case, 
                                                                                                    EMPLOYER_NO_A, 
                                                                                                    EMPLOYER_NO_B)
            
            if result_check_relocate_condition_from_A_to_B["result"] == "normal":
                result.update({
                    "status": "normal", 
                    "message": messages["normal"],
                    "count_abnormal": result_check_relocate_condition_from_A_to_B['count_abnormal'],
                    "total_relocate_day": result_check_relocate_condition_from_A_to_B['total_relocate_day']
                })
                combined_data = pd.concat([result_check_inform_exit["data"], result_check_expire_condition["data"], result_check_relocate_condition_from_A_to_B["data"]], ignore_index=True)
                combined_data["case_code"] = combined_data['status'].apply(lambda x: 'pass' if x == 'normal' else "R2/3")
                result['data'] = combined_data.to_dict(orient="records")
            else:
                result.update({
                    "status": "abnormal", 
                    "message": messages["abnormal_relocate"],
                    "count_abnormal": result_check_relocate_condition_from_A_to_B['count_abnormal'],
                    "total_relocate_day": result_check_relocate_condition_from_A_to_B['total_relocate_day']
                })
                result_check_relocate_condition_from_A_to_B["data"]["case_code"] = "R2/3"
                result_check_expire_condition["data"]["case_code"] = "R2/2"
                result_check_inform_exit["data"]["case_code"] = "R1/2"
                combined_data = pd.concat([result_check_inform_exit["data"], result_check_expire_condition["data"], result_check_relocate_condition_from_A_to_B["data"]], ignore_index=True)
                result['data'] = combined_data.to_dict(orient="records")
        else:
            result.update({
                "status": "abnormal", 
                "message": messages["abnormal_expire"],
                "count_abnormal": result_check_expire_condition['count_abnormal'],
                "total_relocate_day": None
            })
            result_check_expire_condition["data"]["case_code"] = "R2/2"
            result_check_inform_exit["data"]["case_code"] = "R1/2"
            combined_data = pd.concat([result_check_inform_exit["data"], result_check_expire_condition["data"]], ignore_index=True)
            result['data'] = combined_data.to_dict(orient="records")
    else:
        result.update({
            "status": "abnormal",
            "message": messages["abnormal_resign_A"],
            "count_abnormal": result_check_inform_exit['count_abnormal'],
            "total_relocate_day": None
        })
        result_check_inform_exit["data"]["case_code"] = "R2/1"
        result['data'] = result_check_inform_exit["data"].to_dict(orient="records")
    
    return result



In [52]:
data_TC9 = data[data["FORM_ID"].isin(["00015","00016"])]
db_TC9 = db[db["FORM_ID"].isin(["00015","00016"])]

config_case = {
    "number": 20,
    "day": 14
    }

In [53]:
data_TC9

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL
77,2024-06-27,00015,00076,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00
78,2024-06-27,00015,00077,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00
79,2024-06-27,00015,00078,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00
80,2024-06-27,00015,00079,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00
81,2024-06-27,00015,00080,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00
82,2024-06-27,00015,00081,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00
83,2024-06-27,00015,00082,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00
84,2024-06-27,00015,00083,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00
85,2024-06-27,00015,00084,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00
86,2024-06-27,00015,00085,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00


In [54]:
result = flow_2(data_TC9, db_TC9, config_case, db_TC9["EMPLOYER_NO"].unique()[-1], data_TC9["EMPLOYER_NO"].unique()[-1])

   CREATED_TIMESTAMP FORM_ID ALIEN_ID JOB_DESCRIPTION EMPLOYER_NO  \
77        2024-04-27   00015    00076        งานทำมือ    12346800   
78        2024-04-27   00015    00077        งานทำมือ    12346800   
79        2024-04-27   00015    00078        งานทำมือ    12346800   
80        2024-04-27   00015    00079        งานทำมือ    12346800   
81        2024-04-27   00015    00080        งานทำมือ    12346800   
82        2024-04-27   00015    00081        งานทำมือ    12346800   
83        2024-04-27   00015    00082        งานทำมือ    12346800   
84        2024-04-27   00015    00083        งานทำมือ    12346800   
85        2024-04-27   00015    00084        งานทำมือ    12346800   
86        2024-04-27   00015    00085        งานทำมือ    12346800   
87        2024-04-27   00015    00086        งานทำมือ    12346800   
88        2024-04-27   00015    00087        งานทำมือ    12346800   
89        2024-04-30   00016    00088          กรรมกร    12346800   
90        2024-04-30   00016    00

In [55]:
pd.DataFrame(result["data"])

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,status,abnormal_desc,case_code
0,2024-06-27,00015,00076,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,Aliens moved to B exceeding the limit of peopl...,R2/3
1,2024-06-27,00015,00077,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,Aliens moved to B exceeding the limit of peopl...,R2/3
2,2024-06-27,00015,00078,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,Aliens moved to B exceeding the limit of peopl...,R2/3
3,2024-06-27,00015,00079,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,Aliens moved to B exceeding the limit of peopl...,R2/3
4,2024-06-27,00015,00080,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,Aliens moved to B exceeding the limit of peopl...,R2/3
5,2024-06-27,00015,00081,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,Aliens moved to B exceeding the limit of peopl...,R2/3
6,2024-06-27,00015,00082,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,Aliens moved to B exceeding the limit of peopl...,R2/3
7,2024-06-27,00015,00083,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,Aliens moved to B exceeding the limit of peopl...,R2/3
8,2024-06-27,00015,00084,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,Aliens moved to B exceeding the limit of peopl...,R2/3
9,2024-06-27,00015,00085,งานทำมือ,66148977,MT_59,Consider request round1,2027-04-15 11:11:00,abnormal,Aliens moved to B exceeding the limit of peopl...,R2/3


In [103]:
def flow_4(data, db, config_case):
    """
    Flow 4: Evaluate relocation conditions and departure reporting status of aliens.

    Parameters:
    - data (pd.DataFrame): Current dataset containing relocation requests.
    - db (pd.DataFrame): Historical dataset of relocation records.
    - config_case (dict): Configuration specifying the limit on the number of aliens and maximum days for relocation.
        - "number" (int): Maximum allowed number for aliens.
        - "day" (str): limit number move to B
      Example:
        config_case = {
            "number": 50,
            "day": 20
        }

    Returns:
    - dict: A dictionary with the following keys:
        - 'status' (str): Overall status ("normal" or "abnormal").
        - 'message' (str): Description of the status.
        - 'count_abnormal' (int): Count of abnormalities found.
        - 'total_relocate_day' (int): Total days of relocation.
        - 'data' (list of dict): Relevant data for further inspection.
    """
    messages = {
        "normal": "Aliens moved to B not exceeding the limit of people and have been relocated for less than a specified number of days.",
        "abnormal_relocate_B": "Aliens moved to B exceeding the limit of people and have been relocated for more than a specified number of days.",
        "check_inform_exit": "Aliens have not yet reported their departure from the old company but have already applied for the new one."
    }
    
    # Check if aliens have reported their departure
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'total_relocate_day': 0,
        'data': None
    }
    
    if result_check_inform_exit["result"] == "normal":
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        data_case = data[~data['ALIEN_ID'].isin(alien_ids_to_drop)]
        db_case = db[~db['ALIEN_ID'].isin(alien_ids_to_drop)]
        
        # Create concat_df
        data_case_agg = data_case.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
        db_case_agg = db_case.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
        concat_df = pd.concat([db_case_agg, data_case_agg], axis=0).reset_index(drop=True)
        
        result_check_relocate_condition_from_B = check_relocate_condition_from_B(data_case, concat_df, config_case["number"], config_case["day"])
        
        if result_check_relocate_condition_from_B["result"] == "normal":
            result.update({
                "status": "normal", 
                "message": messages["normal"], 
                "count_abnormal": result_check_relocate_condition_from_B['count_abnormal'],
                "total_relocate_day": result_check_relocate_condition_from_B['total_relocate_day'],
                "data": pd.concat([result_check_inform_exit['data'], result_check_relocate_condition_from_B["data"]], ignore_index=True).to_dict(orient='records')
            })
        else:
            result.update({
                "status": "abnormal", 
                "message": messages["abnormal_relocate_B"],
                "count_abnormal": result_check_relocate_condition_from_B['count_abnormal'],
                "total_relocate_day": result_check_relocate_condition_from_B['total_relocate_day'],
                "data": pd.concat([result_check_inform_exit['data'], result_check_relocate_condition_from_B["data"]], ignore_index=True).to_dict(orient='records')
            })
    
    else:
        result.update({
            "status": "abnormal", 
            "message": messages["check_inform_exit"],
            "count_abnormal": result_check_inform_exit['count_abnormal'],
            "total_relocate_day" : None,
            "data": result_check_inform_exit['data'].to_dict(orient='records')
        })
    
    return result

   

In [111]:
def flow_4(data, db, config_case):
    """
    Flow 4: Evaluate relocation conditions and departure reporting status of aliens.

    Parameters:
    - data (pd.DataFrame): Current dataset containing relocation requests.
    - db (pd.DataFrame): Historical dataset of relocation records.
    - config_case (dict): Configuration specifying the limit on the number of aliens and maximum days for relocation.
        - "number" (int): Maximum allowed number for aliens.
        - "day" (str): limit number move to B
      Example:
        config_case = {
            "number": 50,
            "day": 20
        }

    Returns:
    - dict: A dictionary with the following keys:
        - 'status' (str): Overall status ("normal" or "abnormal").
        - 'message' (str): Description of the status.
        - 'count_abnormal' (int): Count of abnormalities found.
        - 'total_relocate_day' (int): Total days of relocation.
        - 'data' (list of dict): Relevant data for further inspection.
    """
    messages = {
        "normal": "Aliens moved to B not exceeding the limit of people and have been relocated for less than a specified number of days.",
        "abnormal_relocate_B": "Aliens moved to B exceeding the limit of people and have been relocated for more than a specified number of days.",
        "check_inform_exit": "Aliens have not yet reported their departure from the old company but have already applied for the new one."
    }
    
    # Check if aliens have reported their departure
    result_check_inform_exit = check_inform_exit(data, db)
    
    # Initialize result dictionary
    result = {
        'status': '',
        'message': '',
        'count_abnormal': 0,
        'total_relocate_day': 0,
        'data': None
    }
    
    if result_check_inform_exit["result"] == "normal":
        alien_ids_to_drop = result_check_inform_exit["data"]['ALIEN_ID'].unique()
        data_case = data[~data['ALIEN_ID'].isin(alien_ids_to_drop)]
        db_case = db[~db['ALIEN_ID'].isin(alien_ids_to_drop)]
        
        # Create concat_df
        data_case_agg = data_case.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
        db_case_agg = db_case.groupby(['CREATED_TIMESTAMP', 'EMPLOYER_NO', 'FORM_ID']).agg(ALIEN_COUNT=('ALIEN_ID', 'count')).reset_index()
        concat_df = pd.concat([db_case_agg, data_case_agg], axis=0).reset_index(drop=True)
        
        result_check_relocate_condition_from_B = check_relocate_condition_from_B(data_case, concat_df, config_case["number"], config_case["day"])
        
        if result_check_relocate_condition_from_B["result"] == "normal":
            result.update({
                "status": "normal", 
                "message": messages["normal"], 
                "count_abnormal": result_check_relocate_condition_from_B['count_abnormal'],
                "total_relocate_day": result_check_relocate_condition_from_B['total_relocate_day']
            })
            combined_data = pd.concat([result_check_inform_exit["data"], result_check_relocate_condition_from_B["data"]], ignore_index=True)
            combined_data["case_code"] = combined_data['status'].apply(lambda x: 'pass' if x == 'normal' else "R4/2")
            result['data'] = combined_data.to_dict(orient='records')
        else:
            result.update({
                "status": "abnormal", 
                "message": messages["abnormal_relocate_B"],
                "count_abnormal": result_check_relocate_condition_from_B['count_abnormal'],
                "total_relocate_day": result_check_relocate_condition_from_B['total_relocate_day']
            })
            result_check_relocate_condition_from_B["data"]["case_code"] = "R4/2"
            result_check_inform_exit["data"]["case_code"] = "R4/1"
            combined_data = pd.concat([result_check_inform_exit["data"], result_check_relocate_condition_from_B["data"]], ignore_index=True)
            result['data'] = combined_data.to_dict(orient='records')
    
    else:
        result.update({
            "status": "abnormal", 
            "message": messages["check_inform_exit"],
            "count_abnormal": result_check_inform_exit['count_abnormal'],
            "total_relocate_day" : None
        })
        result_check_inform_exit["data"]["case_code"] = "R4/1"
        result['data'] = result_check_inform_exit['data'].to_dict(orient='records')
    
    return result


In [120]:
data_TC9 = data[data["FORM_ID"].isin(["00021","00029","00030","00031"])]
db_TC9 = db[db["FORM_ID"].isin(["00021","00029","00030","00031"])]

config_case = {
    "number": 50,
    "day": 20
    }

In [121]:
result = flow_4(data_TC9, db_TC9, config_case)

/opt/project/src/DOE/src/function/flow/module.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered_abnormal['abnormal_desc'] = abnormal_message
/opt/project/src/DOE/src/function/flow/module.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["status"] = "abnormal"
/opt/project/src/DOE/src/function/flow/module.py:239: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [122]:
result["data"]

[{'CREATED_TIMESTAMP': Timestamp('2024-06-27 00:00:00'),
  'FORM_ID': '00021',
  'ALIEN_ID': '00400',
  'JOB_DESCRIPTION': 'กรรมกร',
  'EMPLOYER_NO': '19152919',
  'MASTER_FORM_TYPE': 'MT_59',
  'MASTER_FORM_STATUS': 'Consider request round1',
  'VALID_UNTIL': Timestamp('2027-04-16 11:11:00'),
  'status': 'abnormal',
  'abnormal_desc': 'Aliens have not yet reported their departure from the old company.',
  'case_code': 'R4/1'},
 {'CREATED_TIMESTAMP': Timestamp('2024-07-01 00:00:00'),
  'FORM_ID': '00029',
  'ALIEN_ID': '00238',
  'JOB_DESCRIPTION': 'กรรมกร',
  'EMPLOYER_NO': '23152821',
  'MASTER_FORM_TYPE': 'MT_59',
  'MASTER_FORM_STATUS': 'Consider request round1',
  'VALID_UNTIL': Timestamp('2027-04-24 11:10:59.995000'),
  'status': 'abnormal',
  'abnormal_desc': 'Aliens moved to B exceeding the limit of people and have been relocated for more than a specified number of days.',
  'case_code': 'R4/2'},
 {'CREATED_TIMESTAMP': Timestamp('2024-07-01 00:00:00'),
  'FORM_ID': '00029',
  '

In [123]:
pd.DataFrame(result["data"])

,CREATED_TIMESTAMP,FORM_ID,ALIEN_ID,JOB_DESCRIPTION,EMPLOYER_NO,MASTER_FORM_TYPE,MASTER_FORM_STATUS,VALID_UNTIL,status,abnormal_desc,case_code
0,2024-06-27,00021,00400,กรรมกร,19152919,MT_59,Consider request round1,2027-04-16 11:11:00.000,abnormal,Aliens have not yet reported their departure f...,R4/1
1,2024-07-01,00029,00238,กรรมกร,23152821,MT_59,Consider request round1,2027-04-24 11:10:59.995,abnormal,Aliens moved to B exceeding the limit of peopl...,R4/2
2,2024-07-01,00029,00239,กรรมกร,23152821,MT_59,Consider request round1,2027-04-24 11:10:59.995,abnormal,Aliens moved to B exceeding the limit of peopl...,R4/2
3,2024-07-01,00029,00240,กรรมกร,23152821,MT_59,Consider request round1,2027-04-24 11:10:59.995,abnormal,Aliens moved to B exceeding the limit of peopl...,R4/2
4,2024-07-01,00029,00241,กรรมกร,23152821,MT_59,Consider request round1,2027-04-24 11:10:59.995,abnormal,Aliens moved to B exceeding the limit of peopl...,R4/2
5,2024-07-01,00029,00242,กรรมกร,23152821,MT_59,Consider request round1,2027-04-24 11:10:59.995,abnormal,Aliens moved to B exceeding the limit of peopl...,R4/2
6,2024-07-01,00029,00243,กรรมกร,23152821,MT_59,Consider request round1,2027-04-24 11:10:59.995,abnormal,Aliens moved to B exceeding the limit of peopl...,R4/2
7,2024-07-01,00029,00244,กรรมกร,23152821,MT_59,Consider request round1,2027-04-24 11:10:59.995,abnormal,Aliens moved to B exceeding the limit of peopl...,R4/2
8,2024-07-01,00029,00245,กรรมกร,23152821,MT_59,Consider request round1,2027-04-24 11:10:59.995,abnormal,Aliens moved to B exceeding the limit of peopl...,R4/2
9,2024-07-01,00029,00246,กรรมกร,23152821,MT_59,Consider request round1,2027-04-24 11:10:59.995,abnormal,Aliens moved to B exceeding the limit of peopl...,R4/2
